# 🏗️ Schema Setup & Load 250 Policies

This notebook handles **both** schema creation and policy loading so you can
re-run the demo notebook without it wiping your data each time.

**Workflow:** Run this notebook first → then run the demo notebook (skip Steps 1 & 2).

| Step | What it does |
|------|--------------|
| 1 | Drop & recreate all non-policy tables (claims, payouts, outages, weather, etc.) |
| 2 | Create policies table if it doesn't exist |
| 3 | Load all 250 policies (11 original + 239 new) with deduplication |
| 4 | Verify & summarize |

| Metric | Value |
|--------|-------|
| Total policies | **250** |
| Cities covered | **40** |
| Business types | **33** |
| States/Territories | **30** |


---
## 📐 Step 1 — Create / Reset Schema

This drops and recreates all **non-policy** tables (claims, payouts, outage events, weather, etc.)
so the demo notebook gets a clean slate each run, while **preserving your policies**.

In [ ]:
from datetime import datetime
from pyspark.sql.types import (
    StructType, StructField, StringType, DoubleType, IntegerType, TimestampType
)

now = datetime.utcnow()

# ── Drop non-policy tables (safe to re-run) ──
non_policy_tables = [
    "event_audit_log", "payouts", "claims",
    "weather_data", "social_signals", "outage_events", "outage_raw"
]

print("🗑️  Dropping non-policy tables...")
for table in non_policy_tables:
    spark.sql(f"DROP TABLE IF EXISTS {table}")
    print(f"   Dropped: {table}")

print("\n✅ Non-policy tables cleared. Policies table preserved.")

### Create All Tables (IF NOT EXISTS)

In [ ]:
# -- POLICIES --
spark.sql("""
CREATE TABLE IF NOT EXISTS policies (
    policy_id STRING, business_name STRING, business_type STRING,
    zip_code STRING, address STRING, city STRING, state STRING,
    latitude DOUBLE, longitude DOUBLE,
    threshold_minutes INT, hourly_rate DOUBLE, max_payout DOUBLE,
    status STRING, effective_date TIMESTAMP, expiration_date TIMESTAMP,
    contact_email STRING, contact_phone STRING,
    created_at TIMESTAMP, updated_at TIMESTAMP
) USING DELTA
""")
print("  ✓ policies")

# -- OUTAGE EVENTS --
spark.sql("""
CREATE TABLE IF NOT EXISTS outage_events (
    event_id STRING, utility_name STRING, zip_code STRING,
    city STRING, state STRING, latitude DOUBLE, longitude DOUBLE,
    affected_customers INT, outage_start TIMESTAMP, outage_end TIMESTAMP,
    duration_minutes INT, reported_cause STRING, status STRING,
    data_source STRING, created_at TIMESTAMP, updated_at TIMESTAMP
) USING DELTA
""")
print("  ✓ outage_events")

# -- WEATHER DATA --
spark.sql("""
CREATE TABLE IF NOT EXISTS weather_data (
    weather_id STRING, event_id STRING, zip_code STRING,
    latitude DOUBLE, longitude DOUBLE,
    temperature_f DOUBLE, wind_speed_mph DOUBLE, wind_gust_mph DOUBLE,
    conditions STRING, severe_weather_alert BOOLEAN, alert_type STRING,
    observation_time TIMESTAMP, created_at TIMESTAMP
) USING DELTA
""")
print("  ✓ weather_data")

# -- SOCIAL SIGNALS --
spark.sql("""
CREATE TABLE IF NOT EXISTS social_signals (
    signal_id STRING, platform STRING, text STRING,
    zip_code STRING, latitude DOUBLE, longitude DOUBLE,
    timestamp TIMESTAMP, user_id STRING,
    engagement_count INT, relevance_score DOUBLE,
    event_id STRING, created_at TIMESTAMP
) USING DELTA
""")
print("  ✓ social_signals")

# -- CLAIMS --
spark.sql("""
CREATE TABLE IF NOT EXISTS claims (
    claim_id STRING, policy_id STRING, outage_event_id STRING,
    status STRING, filed_at TIMESTAMP, validated_at TIMESTAMP,
    approved_at TIMESTAMP, denied_at TIMESTAMP, denial_reason STRING,
    payout_amount DOUBLE, ai_confidence_score DOUBLE, ai_reasoning STRING,
    fraud_flags STRING, weather_factor DOUBLE, severity_assessment STRING,
    created_at TIMESTAMP, updated_at TIMESTAMP
) USING DELTA
""")
print("  ✓ claims")

# -- PAYOUTS --
spark.sql("""
CREATE TABLE IF NOT EXISTS payouts (
    payout_id STRING, claim_id STRING, policy_id STRING,
    amount DOUBLE, status STRING,
    initiated_at TIMESTAMP, completed_at TIMESTAMP,
    transaction_id STRING, payment_method STRING, created_at TIMESTAMP
) USING DELTA
""")
print("  ✓ payouts")

# -- RAW STAGING --
spark.sql("""
CREATE TABLE IF NOT EXISTS outage_raw (
    id INT, event_id STRING, utility_name STRING, state STRING,
    affected_customers INT, latitude DOUBLE, longitude DOUBLE,
    zip_code STRING, data_source STRING, raw_json STRING,
    last_updated TIMESTAMP, ingestion_timestamp TIMESTAMP
) USING DELTA
""")
print("  ✓ outage_raw")

# -- EVENT AUDIT LOG --
spark.sql("""
CREATE TABLE IF NOT EXISTS event_audit_log (
    sequence INT,
    event_id STRING,
    event_type STRING,
    subject STRING,
    event_time STRING,
    data_summary STRING,
    status STRING,
    error STRING,
    created_at TIMESTAMP
) USING DELTA
""")
print("  ✓ event_audit_log")

print("\n✅ All 8 tables ready (policies preserved, 7 others recreated).")

---
## 📑 Step 2 — Load 250 Policies

Includes the original 11 policies (BI-001 → BI-011) plus 239 new ones (BI-012 → BI-250).
Uses deduplication so this is safe to re-run — existing policies won't be duplicated.

In [ ]:
policy_schema = StructType([
    StructField("policy_id", StringType()),
    StructField("business_name", StringType()),
    StructField("business_type", StringType()),
    StructField("zip_code", StringType()),
    StructField("address", StringType()),
    StructField("city", StringType()),
    StructField("state", StringType()),
    StructField("latitude", DoubleType()),
    StructField("longitude", DoubleType()),
    StructField("threshold_minutes", IntegerType()),
    StructField("hourly_rate", DoubleType()),
    StructField("max_payout", DoubleType()),
    StructField("status", StringType()),
    StructField("effective_date", TimestampType()),
    StructField("expiration_date", TimestampType()),
    StructField("contact_email", StringType()),
    StructField("contact_phone", StringType()),
    StructField("created_at", TimestampType()),
    StructField("updated_at", TimestampType()),
])

print("✅ Policy schema defined.")

---
### 🏠 Original 11 Policies (BI-001 → BI-011)

These are the original demo policies across Seattle, Portland, San Francisco, Los Angeles, and New York.

In [ ]:
original_policies = [
    {"policy_id": "BI-001", "business_name": "Pike Place Coffee Co", "business_type": "Coffee Shop",
     "zip_code": "98101", "address": "123 Pike St", "city": "Seattle", "state": "WA",
     "latitude": 47.6097, "longitude": -122.3425, "threshold_minutes": 120,
     "hourly_rate": 500.0, "max_payout": 10000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@pikeplacecoffee.com", "contact_phone": "206-555-0101",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-002", "business_name": "Broadway Restaurant & Bar", "business_type": "Restaurant",
     "zip_code": "98102", "address": "456 Broadway Ave E", "city": "Seattle", "state": "WA",
     "latitude": 47.6234, "longitude": -122.3212, "threshold_minutes": 60,
     "hourly_rate": 750.0, "max_payout": 15000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@broadwayrestaurant.com", "contact_phone": "206-555-0102",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-003", "business_name": "Capitol Hill Fitness Center", "business_type": "Gym",
     "zip_code": "98102", "address": "789 E Pine St", "city": "Seattle", "state": "WA",
     "latitude": 47.6145, "longitude": -122.3201, "threshold_minutes": 180,
     "hourly_rate": 300.0, "max_payout": 8000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@capitolhillfitness.com", "contact_phone": "206-555-0103",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-004", "business_name": "Downtown Portland Bakery", "business_type": "Bakery",
     "zip_code": "97201", "address": "321 SW Morrison St", "city": "Portland", "state": "OR",
     "latitude": 45.5202, "longitude": -122.6742, "threshold_minutes": 90,
     "hourly_rate": 600.0, "max_payout": 12000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "baker@portlandbakery.com", "contact_phone": "503-555-0104",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-005", "business_name": "Pearl District Co-Working", "business_type": "Co-Working Space",
     "zip_code": "97209", "address": "555 NW 13th Ave", "city": "Portland", "state": "OR",
     "latitude": 45.5276, "longitude": -122.6847, "threshold_minutes": 240,
     "hourly_rate": 400.0, "max_payout": 20000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@pearlcowork.com", "contact_phone": "503-555-0105",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-006", "business_name": "Mission District Taqueria", "business_type": "Restaurant",
     "zip_code": "94110", "address": "2800 Mission St", "city": "San Francisco", "state": "CA",
     "latitude": 37.7516, "longitude": -122.4180, "threshold_minutes": 90,
     "hourly_rate": 550.0, "max_payout": 11000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hola@missiontaqueria.com", "contact_phone": "415-555-0106",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-007", "business_name": "SOMA Tech Startup Hub", "business_type": "Co-Working Space",
     "zip_code": "94105", "address": "100 Howard St", "city": "San Francisco", "state": "CA",
     "latitude": 37.7912, "longitude": -122.3953, "threshold_minutes": 60,
     "hourly_rate": 1200.0, "max_payout": 30000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@somatechhub.com", "contact_phone": "415-555-0107",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-008", "business_name": "Venice Beach Surf Shop", "business_type": "Retail",
     "zip_code": "90291", "address": "1501 Ocean Front Walk", "city": "Los Angeles", "state": "CA",
     "latitude": 33.9850, "longitude": -118.4695, "threshold_minutes": 120,
     "hourly_rate": 350.0, "max_payout": 8000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "shop@venicesurfshop.com", "contact_phone": "310-555-0108",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-009", "business_name": "Hollywood Production Studio", "business_type": "Production Studio",
     "zip_code": "90028", "address": "6500 Sunset Blvd", "city": "Los Angeles", "state": "CA",
     "latitude": 34.0983, "longitude": -118.3267, "threshold_minutes": 60,
     "hourly_rate": 1500.0, "max_payout": 40000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "production@hollywoodstudio.com", "contact_phone": "323-555-0109",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-010", "business_name": "Manhattan Fine Dining", "business_type": "Restaurant",
     "zip_code": "10022", "address": "300 Park Ave", "city": "New York", "state": "NY",
     "latitude": 40.7614, "longitude": -73.9776, "threshold_minutes": 45,
     "hourly_rate": 1200.0, "max_payout": 30000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "reservations@manhattanfine.com", "contact_phone": "212-555-0110",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-011", "business_name": "Brooklyn Artisan Market", "business_type": "Retail",
     "zip_code": "11211", "address": "200 Bedford Ave", "city": "Brooklyn", "state": "NY",
     "latitude": 40.7181, "longitude": -73.9571, "threshold_minutes": 120,
     "hourly_rate": 400.0, "max_payout": 9000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "market@brooklynartisan.com", "contact_phone": "718-555-0111",
     "created_at": now, "updated_at": now},
]
print(f"  🏠 Original policies: {len(original_policies)} defined")

---
### 🏙️ 239 New Policies by City (BI-012 → BI-250)

Each cell below defines policies for one city. They are all combined and written in a single batch at the end.

### 🌧️ Seattle, WA — 6 Policies

In [ ]:
seattle_policies = [
    {"policy_id": "BI-012", "business_name": "Capitol Hill Coffee Roasters", "business_type": "Coffee Shop",
     "zip_code": "98101", "address": "3250 Lincoln Ave", "city": "Seattle", "state": "WA",
     "latitude": 47.6126, "longitude": -122.3128, "threshold_minutes": 138,
     "hourly_rate": 275.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@chc.com", "contact_phone": "206-555-5741",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-013", "business_name": "Fremont Pharmacy & Wellness", "business_type": "Pharmacy",
     "zip_code": "98102", "address": "2764 E Commerce Ln", "city": "Seattle", "state": "WA",
     "latitude": 47.6002, "longitude": -122.3344, "threshold_minutes": 50,
     "hourly_rate": 950.0, "max_payout": 20500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@fremontpharmacywellness.com", "contact_phone": "206-555-2169",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-014", "business_name": "Ballard Yoga & Pilates Studio", "business_type": "Fitness Studio",
     "zip_code": "98103", "address": "9225 Lake Ln", "city": "Seattle", "state": "WA",
     "latitude": 47.5934, "longitude": -122.334, "threshold_minutes": 197,
     "hourly_rate": 325.0, "max_payout": 6500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@bys.com", "contact_phone": "206-555-1525",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-015", "business_name": "Queen Anne Seafood Grill", "business_type": "Restaurant",
     "zip_code": "98104", "address": "3583 NE Washington Rd", "city": "Seattle", "state": "WA",
     "latitude": 47.6269, "longitude": -122.3287, "threshold_minutes": 65,
     "hourly_rate": 800.0, "max_payout": 13000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@qas.com", "contact_phone": "206-555-3340",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-016", "business_name": "Wallingford Urgent Care Clinic", "business_type": "Medical Office",
     "zip_code": "98105", "address": "9660 E Union Ln", "city": "Seattle", "state": "WA",
     "latitude": 47.6081, "longitude": -122.3197, "threshold_minutes": 38,
     "hourly_rate": 1000.0, "max_payout": 35000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@wuc.com", "contact_phone": "206-555-9348",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-017", "business_name": "University District Live Music Venue", "business_type": "Entertainment Venue",
     "zip_code": "98109", "address": "7016 NE Queen Ave", "city": "Seattle", "state": "WA",
     "latitude": 47.5867, "longitude": -122.3257, "threshold_minutes": 76,
     "hourly_rate": 875.0, "max_payout": 19000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@udl.com", "contact_phone": "206-555-8668",
     "created_at": now, "updated_at": now}
]
print(f"  🌧️ Seattle, WA: {len(seattle_policies)} policies defined")

### 🌲 Portland, OR — 6 Policies

In [ ]:
portland_policies = [
    {"policy_id": "BI-018", "business_name": "Pearl District Pizza Kitchen", "business_type": "Restaurant",
     "zip_code": "97201", "address": "9107 SE Cedar Way", "city": "Portland", "state": "OR",
     "latitude": 45.5335, "longitude": -122.6928, "threshold_minutes": 112,
     "hourly_rate": 425.0, "max_payout": 13500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@pdp.com", "contact_phone": "503-555-4470",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-019", "business_name": "Alberta Bookshop", "business_type": "Retail",
     "zip_code": "97202", "address": "6218 SW State Ct", "city": "Portland", "state": "OR",
     "latitude": 45.5053, "longitude": -122.6836, "threshold_minutes": 298,
     "hourly_rate": 300.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@albertabookshop.com", "contact_phone": "503-555-5061",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-020", "business_name": "Hawthorne Boutique", "business_type": "Retail",
     "zip_code": "97204", "address": "1064 Lake Ave", "city": "Portland", "state": "OR",
     "latitude": 45.5229, "longitude": -122.6698, "threshold_minutes": 148,
     "hourly_rate": 325.0, "max_payout": 6000.0, "status": "pending",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@hawthorneboutique.com", "contact_phone": "503-555-2160",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-021", "business_name": "Division Movie Theater", "business_type": "Entertainment Venue",
     "zip_code": "97205", "address": "9455 E King Rd", "city": "Portland", "state": "OR",
     "latitude": 45.5059, "longitude": -122.6926, "threshold_minutes": 92,
     "hourly_rate": 850.0, "max_payout": 25000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@dmt.com", "contact_phone": "503-555-7669",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-022", "business_name": "Mississippi Artisan Bakery", "business_type": "Bakery",
     "zip_code": "97209", "address": "987 Jefferson Ave", "city": "Portland", "state": "OR",
     "latitude": 45.5129, "longitude": -122.6786, "threshold_minutes": 72,
     "hourly_rate": 425.0, "max_payout": 13500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@mab.com", "contact_phone": "503-555-6559",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-023", "business_name": "Sellwood Boutique Hotel", "business_type": "Hotel",
     "zip_code": "97210", "address": "4663 N State Dr", "city": "Portland", "state": "OR",
     "latitude": 45.5047, "longitude": -122.6768, "threshold_minutes": 55,
     "hourly_rate": 2325.0, "max_payout": 37500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@sbh.com", "contact_phone": "503-555-2604",
     "created_at": now, "updated_at": now}
]
print(f"  🌲 Portland, OR: {len(portland_policies)} policies defined")

### 🌉 San Francisco, CA — 6 Policies

In [ ]:
san_francisco_policies = [
    {"policy_id": "BI-024", "business_name": "Mission Dental Practice", "business_type": "Dental Office",
     "zip_code": "94102", "address": "751 NE Queen Ave", "city": "San Francisco", "state": "CA",
     "latitude": 37.797, "longitude": -122.4159, "threshold_minutes": 88,
     "hourly_rate": 1125.0, "max_payout": 21000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@mdp.com", "contact_phone": "415-555-5272",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-025", "business_name": "SOMA Event Space", "business_type": "Entertainment Venue",
     "zip_code": "94103", "address": "7591 Market Ave", "city": "San Francisco", "state": "CA",
     "latitude": 37.7632, "longitude": -122.4379, "threshold_minutes": 73,
     "hourly_rate": 1000.0, "max_payout": 18500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@ses.com", "contact_phone": "415-555-2638",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-026", "business_name": "Hayes Valley Boutique", "business_type": "Retail",
     "zip_code": "94104", "address": "4102 S Commerce Way", "city": "San Francisco", "state": "CA",
     "latitude": 37.7565, "longitude": -122.4269, "threshold_minutes": 129,
     "hourly_rate": 450.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@hvb.com", "contact_phone": "415-555-9900",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-027", "business_name": "North Beach Manufacturing Works", "business_type": "Manufacturing",
     "zip_code": "94105", "address": "9186 S Hill Ave", "city": "San Francisco", "state": "CA",
     "latitude": 37.7903, "longitude": -122.418, "threshold_minutes": 90,
     "hourly_rate": 1500.0, "max_payout": 54500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@nbm.com", "contact_phone": "415-555-2891",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-028", "business_name": "Noe Valley Tutoring Center", "business_type": "Education",
     "zip_code": "94107", "address": "4425 W High Rd", "city": "San Francisco", "state": "CA",
     "latitude": 37.764, "longitude": -122.4339, "threshold_minutes": 103,
     "hourly_rate": 475.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@nvt.com", "contact_phone": "415-555-2512",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-029", "business_name": "Marina Juice & Smoothie Bar", "business_type": "Juice Bar",
     "zip_code": "94108", "address": "4391 SE Cedar Rd", "city": "San Francisco", "state": "CA",
     "latitude": 37.7501, "longitude": -122.4058, "threshold_minutes": 201,
     "hourly_rate": 275.0, "max_payout": 4000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@mjb.com", "contact_phone": "415-555-1158",
     "created_at": now, "updated_at": now}
]
print(f"  🌉 San Francisco, CA: {len(san_francisco_policies)} policies defined")

### 🎬 Los Angeles, CA — 6 Policies

In [ ]:
los_angeles_policies = [
    {"policy_id": "BI-030", "business_name": "Silver Lake Cold Storage Facility", "business_type": "Cold Storage",
     "zip_code": "90001", "address": "5478 W River Ave", "city": "Los Angeles", "state": "CA",
     "latitude": 34.0386, "longitude": -118.2675, "threshold_minutes": 26,
     "hourly_rate": 2450.0, "max_payout": 82000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@slc.com", "contact_phone": "213-555-9346",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-031", "business_name": "Echo Park Insurance Agency", "business_type": "Financial Services",
     "zip_code": "90012", "address": "4379 Cedar Way", "city": "Los Angeles", "state": "CA",
     "latitude": 34.0438, "longitude": -118.2673, "threshold_minutes": 70,
     "hourly_rate": 1275.0, "max_payout": 32500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@epi.com", "contact_phone": "213-555-8119",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-032", "business_name": "Venice Co-Working Hub", "business_type": "Co-Working Space",
     "zip_code": "90013", "address": "9546 Park Way", "city": "Los Angeles", "state": "CA",
     "latitude": 34.0575, "longitude": -118.2431, "threshold_minutes": 134,
     "hourly_rate": 625.0, "max_payout": 13000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@vch.com", "contact_phone": "213-555-1027",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-033", "business_name": "Downtown Movie Theater", "business_type": "Entertainment Venue",
     "zip_code": "90015", "address": "6067 NW Liberty Ave", "city": "Los Angeles", "state": "CA",
     "latitude": 34.0632, "longitude": -118.2316, "threshold_minutes": 93,
     "hourly_rate": 1250.0, "max_payout": 23000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@dmt.com", "contact_phone": "213-555-3041",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-034", "business_name": "Los Feliz Bookshop", "business_type": "Retail",
     "zip_code": "90017", "address": "9183 NE Maple Dr", "city": "Los Angeles", "state": "CA",
     "latitude": 34.0605, "longitude": -118.2524, "threshold_minutes": 272,
     "hourly_rate": 375.0, "max_payout": 6000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@lfb.com", "contact_phone": "213-555-3851",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-035", "business_name": "Arts District Pet Supply Store", "business_type": "Pet Services",
     "zip_code": "90019", "address": "9602 SW Queen Ln", "city": "Los Angeles", "state": "CA",
     "latitude": 34.0689, "longitude": -118.2535, "threshold_minutes": 159,
     "hourly_rate": 375.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@adp.com", "contact_phone": "213-555-8244",
     "created_at": now, "updated_at": now}
]
print(f"  🎬 Los Angeles, CA: {len(los_angeles_policies)} policies defined")

### 🗽 New York, NY — 6 Policies

In [ ]:
new_york_policies = [
    {"policy_id": "BI-036", "business_name": "SoHo Artisan Bakery", "business_type": "Bakery",
     "zip_code": "10001", "address": "7351 NE Queen Ct", "city": "New York", "state": "NY",
     "latitude": 40.7157, "longitude": -74.0151, "threshold_minutes": 111,
     "hourly_rate": 575.0, "max_payout": 12000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@sab.com", "contact_phone": "212-555-3607",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-037", "business_name": "Tribeca Insurance Agency", "business_type": "Financial Services",
     "zip_code": "10003", "address": "8039 W Washington Dr", "city": "New York", "state": "NY",
     "latitude": 40.7298, "longitude": -74.0171, "threshold_minutes": 75,
     "hourly_rate": 1125.0, "max_payout": 31000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@tia.com", "contact_phone": "212-555-2269",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-038", "business_name": "Williamsburg Live Music Venue", "business_type": "Entertainment Venue",
     "zip_code": "10010", "address": "3888 E Union Blvd", "city": "New York", "state": "NY",
     "latitude": 40.7038, "longitude": -74.004, "threshold_minutes": 90,
     "hourly_rate": 1000.0, "max_payout": 22500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@wlm.com", "contact_phone": "212-555-7797",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-039", "business_name": "Harlem Day Spa & Salon", "business_type": "Salon",
     "zip_code": "10013", "address": "9669 NE Lincoln St", "city": "New York", "state": "NY",
     "latitude": 40.6909, "longitude": -73.9894, "threshold_minutes": 142,
     "hourly_rate": 450.0, "max_payout": 10500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@hds.com", "contact_phone": "212-555-1096",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-040", "business_name": "Chelsea Pharmacy & Wellness", "business_type": "Pharmacy",
     "zip_code": "10016", "address": "9047 SW Franklin Dr", "city": "New York", "state": "NY",
     "latitude": 40.7324, "longitude": -73.9892, "threshold_minutes": 67,
     "hourly_rate": 725.0, "max_payout": 16000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@chelseapharmacywellness.com", "contact_phone": "212-555-5471",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-041", "business_name": "Astoria Comedy Club", "business_type": "Entertainment Venue",
     "zip_code": "10019", "address": "7757 SE Monroe Blvd", "city": "New York", "state": "NY",
     "latitude": 40.7212, "longitude": -73.9911, "threshold_minutes": 117,
     "hourly_rate": 700.0, "max_payout": 14000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@acc.com", "contact_phone": "212-555-7455",
     "created_at": now, "updated_at": now}
]
print(f"  🗽 New York, NY: {len(new_york_policies)} policies defined")

### 🏙️ Chicago, IL — 6 Policies

In [ ]:
chicago_policies = [
    {"policy_id": "BI-042", "business_name": "Wicker Park Print Shop", "business_type": "Print Shop",
     "zip_code": "60601", "address": "1871 W King St", "city": "Chicago", "state": "IL",
     "latitude": 41.8588, "longitude": -87.6138, "threshold_minutes": 167,
     "hourly_rate": 575.0, "max_payout": 14000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@wpp.com", "contact_phone": "312-555-7498",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-043", "business_name": "Logan Square Bookshop", "business_type": "Retail",
     "zip_code": "60602", "address": "5041 Grant Ave", "city": "Chicago", "state": "IL",
     "latitude": 41.8876, "longitude": -87.6234, "threshold_minutes": 300,
     "hourly_rate": 350.0, "max_payout": 4500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@lsb.com", "contact_phone": "312-555-9728",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-044", "business_name": "Pilsen Manufacturing Works", "business_type": "Manufacturing",
     "zip_code": "60607", "address": "8131 NW Pine Pl", "city": "Chicago", "state": "IL",
     "latitude": 41.8855, "longitude": -87.6542, "threshold_minutes": 72,
     "hourly_rate": 1850.0, "max_payout": 30000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@pmw.com", "contact_phone": "312-555-3506",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-045", "business_name": "Lakeview Boutique", "business_type": "Retail",
     "zip_code": "60611", "address": "8021 W State Pl", "city": "Chicago", "state": "IL",
     "latitude": 41.8856, "longitude": -87.624, "threshold_minutes": 175,
     "hourly_rate": 350.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@lakeviewboutique.com", "contact_phone": "312-555-5022",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-046", "business_name": "Hyde Park Ice Cream Parlor", "business_type": "Ice Cream Shop",
     "zip_code": "60614", "address": "6309 SW Market St", "city": "Chicago", "state": "IL",
     "latitude": 41.8756, "longitude": -87.6173, "threshold_minutes": 113,
     "hourly_rate": 375.0, "max_payout": 5000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@hpi.com", "contact_phone": "312-555-3979",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-047", "business_name": "Andersonville Tech Incubator", "business_type": "Co-Working Space",
     "zip_code": "60618", "address": "4626 SE Broad St", "city": "Chicago", "state": "IL",
     "latitude": 41.8701, "longitude": -87.6108, "threshold_minutes": 91,
     "hourly_rate": 575.0, "max_payout": 20000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@ati.com", "contact_phone": "312-555-2402",
     "created_at": now, "updated_at": now}
]
print(f"  🏙️ Chicago, IL: {len(chicago_policies)} policies defined")

### 🌴 Miami, FL — 6 Policies

In [ ]:
miami_policies = [
    {"policy_id": "BI-048", "business_name": "Wynwood Boutique", "business_type": "Retail",
     "zip_code": "33125", "address": "2168 N Washington Rd", "city": "Miami", "state": "FL",
     "latitude": 25.7504, "longitude": -80.1681, "threshold_minutes": 158,
     "hourly_rate": 300.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@wynwoodboutique.com", "contact_phone": "305-555-5658",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-049", "business_name": "Brickell Live Music Venue", "business_type": "Entertainment Venue",
     "zip_code": "33127", "address": "1969 NE Hamilton Ave", "city": "Miami", "state": "FL",
     "latitude": 25.785, "longitude": -80.2042, "threshold_minutes": 75,
     "hourly_rate": 1125.0, "max_payout": 24000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@blm.com", "contact_phone": "305-555-2213",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-050", "business_name": "Little Havana Accounting Firm", "business_type": "Financial Services",
     "zip_code": "33130", "address": "4166 NE Pine Ct", "city": "Miami", "state": "FL",
     "latitude": 25.7442, "longitude": -80.1887, "threshold_minutes": 48,
     "hourly_rate": 1825.0, "max_payout": 29500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@lha.com", "contact_phone": "305-555-9561",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-051", "business_name": "Coconut Grove Manufacturing Works", "business_type": "Manufacturing",
     "zip_code": "33131", "address": "1086 E Queen Ave", "city": "Miami", "state": "FL",
     "latitude": 25.7797, "longitude": -80.1673, "threshold_minutes": 69,
     "hourly_rate": 1650.0, "max_payout": 40000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@cgm.com", "contact_phone": "305-555-5335",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-052", "business_name": "Design District Pizza Kitchen", "business_type": "Restaurant",
     "zip_code": "33132", "address": "9829 Church Way", "city": "Miami", "state": "FL",
     "latitude": 25.7643, "longitude": -80.209, "threshold_minutes": 102,
     "hourly_rate": 425.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@ddp.com", "contact_phone": "305-555-5720",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-053", "business_name": "Edgewater Movie Theater", "business_type": "Entertainment Venue",
     "zip_code": "33133", "address": "9763 NE Jefferson Dr", "city": "Miami", "state": "FL",
     "latitude": 25.7403, "longitude": -80.2051, "threshold_minutes": 105,
     "hourly_rate": 875.0, "max_payout": 28000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@emt.com", "contact_phone": "305-555-9922",
     "created_at": now, "updated_at": now}
]
print(f"  🌴 Miami, FL: {len(miami_policies)} policies defined")

### 🤠 Houston, TX — 6 Policies

In [ ]:
houston_policies = [
    {"policy_id": "BI-054", "business_name": "Montrose Craft Brewery", "business_type": "Brewery",
     "zip_code": "77002", "address": "4822 SE Jefferson Pl", "city": "Houston", "state": "TX",
     "latitude": 29.7783, "longitude": -95.3783, "threshold_minutes": 142,
     "hourly_rate": 500.0, "max_payout": 13500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@mcb.com", "contact_phone": "713-555-8451",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-055", "business_name": "The Heights Manufacturing Works", "business_type": "Manufacturing",
     "zip_code": "77003", "address": "118 SW Jefferson Ct", "city": "Houston", "state": "TX",
     "latitude": 29.7839, "longitude": -95.3746, "threshold_minutes": 50,
     "hourly_rate": 1525.0, "max_payout": 36500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@thm.com", "contact_phone": "713-555-1888",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-056", "business_name": "Midtown Florist & Garden", "business_type": "Retail",
     "zip_code": "77004", "address": "9098 SW Maple Way", "city": "Houston", "state": "TX",
     "latitude": 29.7603, "longitude": -95.3727, "threshold_minutes": 72,
     "hourly_rate": 475.0, "max_payout": 10500.0, "status": "expired",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@midtownfloristgarden.com", "contact_phone": "713-555-2989",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-057", "business_name": "East End Architecture Studio", "business_type": "Professional Services",
     "zip_code": "77006", "address": "9148 E Liberty Ave", "city": "Houston", "state": "TX",
     "latitude": 29.7474, "longitude": -95.3869, "threshold_minutes": 61,
     "hourly_rate": 1075.0, "max_payout": 20500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@eea.com", "contact_phone": "713-555-8560",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-058", "business_name": "Museum District Taqueria", "business_type": "Restaurant",
     "zip_code": "77007", "address": "6907 SW Hamilton Rd", "city": "Houston", "state": "TX",
     "latitude": 29.782, "longitude": -95.3802, "threshold_minutes": 105,
     "hourly_rate": 450.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@mdt.com", "contact_phone": "713-555-8483",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-059", "business_name": "Rice Village Print Shop", "business_type": "Print Shop",
     "zip_code": "77008", "address": "1243 NW River Pl", "city": "Houston", "state": "TX",
     "latitude": 29.7654, "longitude": -95.3575, "threshold_minutes": 160,
     "hourly_rate": 450.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@rvp.com", "contact_phone": "713-555-1042",
     "created_at": now, "updated_at": now}
]
print(f"  🤠 Houston, TX: {len(houston_policies)} policies defined")

### 🏔️ Denver, CO — 6 Policies

In [ ]:
denver_policies = [
    {"policy_id": "BI-060", "business_name": "LoDo Craft Brewery", "business_type": "Brewery",
     "zip_code": "80202", "address": "8842 SW Union Ln", "city": "Denver", "state": "CO",
     "latitude": 39.7182, "longitude": -104.9989, "threshold_minutes": 133,
     "hourly_rate": 575.0, "max_payout": 12000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@lcb.com", "contact_phone": "303-555-2121",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-061", "business_name": "RiNo Recording Studio", "business_type": "Recording Studio",
     "zip_code": "80203", "address": "1712 SW Adams Ave", "city": "Denver", "state": "CO",
     "latitude": 39.7641, "longitude": -104.978, "threshold_minutes": 37,
     "hourly_rate": 1625.0, "max_payout": 48000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@rrs.com", "contact_phone": "303-555-5906",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-062", "business_name": "Capitol Hill Vintage Clothing", "business_type": "Retail",
     "zip_code": "80204", "address": "4100 S High Pl", "city": "Denver", "state": "CO",
     "latitude": 39.717, "longitude": -104.9974, "threshold_minutes": 235,
     "hourly_rate": 250.0, "max_payout": 6500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@chv.com", "contact_phone": "303-555-3868",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-063", "business_name": "Cherry Creek Art Gallery", "business_type": "Art Gallery",
     "zip_code": "80205", "address": "3904 SW State Way", "city": "Denver", "state": "CO",
     "latitude": 39.7484, "longitude": -104.9904, "threshold_minutes": 130,
     "hourly_rate": 400.0, "max_payout": 7500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@cca.com", "contact_phone": "303-555-1153",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-064", "business_name": "Highland Juice & Smoothie Bar", "business_type": "Juice Bar",
     "zip_code": "80206", "address": "5761 NE King Way", "city": "Denver", "state": "CO",
     "latitude": 39.7481, "longitude": -105.0074, "threshold_minutes": 235,
     "hourly_rate": 325.0, "max_payout": 6500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@hjb.com", "contact_phone": "303-555-8484",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-065", "business_name": "Baker Taqueria", "business_type": "Restaurant",
     "zip_code": "80209", "address": "9517 Hill Way", "city": "Denver", "state": "CO",
     "latitude": 39.7384, "longitude": -105.0034, "threshold_minutes": 128,
     "hourly_rate": 350.0, "max_payout": 8500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@bakertaqueria.com", "contact_phone": "303-555-5497",
     "created_at": now, "updated_at": now}
]
print(f"  🏔️ Denver, CO: {len(denver_policies)} policies defined")

### 🦞 Boston, MA — 6 Policies

In [ ]:
boston_policies = [
    {"policy_id": "BI-066", "business_name": "Back Bay Architecture Studio", "business_type": "Professional Services",
     "zip_code": "02108", "address": "6342 N Hamilton Ln", "city": "Boston", "state": "MA",
     "latitude": 42.3535, "longitude": -71.0464, "threshold_minutes": 60,
     "hourly_rate": 725.0, "max_payout": 14500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@bba.com", "contact_phone": "617-555-8724",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-067", "business_name": "South End Day Spa & Salon", "business_type": "Salon",
     "zip_code": "02109", "address": "7706 NE Monroe Way", "city": "Boston", "state": "MA",
     "latitude": 42.3515, "longitude": -71.0534, "threshold_minutes": 93,
     "hourly_rate": 525.0, "max_payout": 14500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@sed.com", "contact_phone": "617-555-3290",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-068", "business_name": "Beacon Hill Art Gallery", "business_type": "Art Gallery",
     "zip_code": "02110", "address": "2322 NW Union Rd", "city": "Boston", "state": "MA",
     "latitude": 42.3597, "longitude": -71.079, "threshold_minutes": 125,
     "hourly_rate": 475.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@bha.com", "contact_phone": "617-555-3531",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-069", "business_name": "Allston Insurance Agency", "business_type": "Financial Services",
     "zip_code": "02111", "address": "6170 SW Park Rd", "city": "Boston", "state": "MA",
     "latitude": 42.3659, "longitude": -71.0369, "threshold_minutes": 71,
     "hourly_rate": 1250.0, "max_payout": 32000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@aia.com", "contact_phone": "617-555-6942",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-070", "business_name": "Jamaica Plain Daycare Center", "business_type": "Childcare Center",
     "zip_code": "02113", "address": "4620 SW Park Ave", "city": "Boston", "state": "MA",
     "latitude": 42.3802, "longitude": -71.0517, "threshold_minutes": 33,
     "hourly_rate": 1000.0, "max_payout": 16500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@jpd.com", "contact_phone": "617-555-4475",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-071", "business_name": "Charlestown Comedy Club", "business_type": "Entertainment Venue",
     "zip_code": "02114", "address": "9350 SE Park Ave", "city": "Boston", "state": "MA",
     "latitude": 42.3376, "longitude": -71.0672, "threshold_minutes": 131,
     "hourly_rate": 750.0, "max_payout": 19500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@ccc.com", "contact_phone": "617-555-4538",
     "created_at": now, "updated_at": now}
]
print(f"  🦞 Boston, MA: {len(boston_policies)} policies defined")

### 🍑 Atlanta, GA — 6 Policies

In [ ]:
atlanta_policies = [
    {"policy_id": "BI-072", "business_name": "Midtown Co-Working Hub", "business_type": "Co-Working Space",
     "zip_code": "30303", "address": "2167 NW River Ct", "city": "Atlanta", "state": "GA",
     "latitude": 33.769, "longitude": -84.3969, "threshold_minutes": 152,
     "hourly_rate": 625.0, "max_payout": 14500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@mch.com", "contact_phone": "404-555-9692",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-073", "business_name": "Buckhead Craft Brewery", "business_type": "Brewery",
     "zip_code": "30305", "address": "9133 NE Madison Dr", "city": "Atlanta", "state": "GA",
     "latitude": 33.729, "longitude": -84.3779, "threshold_minutes": 127,
     "hourly_rate": 525.0, "max_payout": 13000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@bcb.com", "contact_phone": "404-555-4743",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-074", "business_name": "Virginia-Highland Marketing Agency", "business_type": "Professional Services",
     "zip_code": "30306", "address": "4276 SW Commerce Blvd", "city": "Atlanta", "state": "GA",
     "latitude": 33.7448, "longitude": -84.4082, "threshold_minutes": 86,
     "hourly_rate": 625.0, "max_payout": 14500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@vma.com", "contact_phone": "404-555-2494",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-075", "business_name": "Decatur Pet Supply Store", "business_type": "Pet Services",
     "zip_code": "30307", "address": "1082 SE King Ct", "city": "Atlanta", "state": "GA",
     "latitude": 33.7612, "longitude": -84.3944, "threshold_minutes": 173,
     "hourly_rate": 275.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@dps.com", "contact_phone": "404-555-3002",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-076", "business_name": "Inman Park Taqueria", "business_type": "Restaurant",
     "zip_code": "30308", "address": "4812 NW Queen Way", "city": "Atlanta", "state": "GA",
     "latitude": 33.7615, "longitude": -84.3696, "threshold_minutes": 142,
     "hourly_rate": 400.0, "max_payout": 10000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@ipt.com", "contact_phone": "404-555-9313",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-077", "business_name": "Old Fourth Ward Print Shop", "business_type": "Print Shop",
     "zip_code": "30309", "address": "1982 E Adams Way", "city": "Atlanta", "state": "GA",
     "latitude": 33.7664, "longitude": -84.3955, "threshold_minutes": 117,
     "hourly_rate": 450.0, "max_payout": 12000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@ofw.com", "contact_phone": "404-555-1430",
     "created_at": now, "updated_at": now}
]
print(f"  🍑 Atlanta, GA: {len(atlanta_policies)} policies defined")

### 🌵 Phoenix, AZ — 6 Policies

In [ ]:
phoenix_policies = [
    {"policy_id": "BI-078", "business_name": "Downtown Fitness Center", "business_type": "Gym",
     "zip_code": "85003", "address": "9919 E Oak St", "city": "Phoenix", "state": "AZ",
     "latitude": 33.4477, "longitude": -112.0656, "threshold_minutes": 134,
     "hourly_rate": 375.0, "max_payout": 11500.0, "status": "expired",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@dfc.com", "contact_phone": "602-555-4460",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-079", "business_name": "Arcadia Vintage Clothing", "business_type": "Retail",
     "zip_code": "85004", "address": "2978 SE Maple Pl", "city": "Phoenix", "state": "AZ",
     "latitude": 33.4398, "longitude": -112.0986, "threshold_minutes": 218,
     "hourly_rate": 275.0, "max_payout": 6500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@avc.com", "contact_phone": "602-555-9196",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-080", "business_name": "Roosevelt Row Tech Incubator", "business_type": "Co-Working Space",
     "zip_code": "85006", "address": "7632 NW Monroe Ave", "city": "Phoenix", "state": "AZ",
     "latitude": 33.427, "longitude": -112.0559, "threshold_minutes": 95,
     "hourly_rate": 825.0, "max_payout": 20000.0, "status": "suspended",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@rrt.com", "contact_phone": "602-555-7626",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-081", "business_name": "Scottsdale Movie Theater", "business_type": "Entertainment Venue",
     "zip_code": "85007", "address": "7638 N Hamilton Ln", "city": "Phoenix", "state": "AZ",
     "latitude": 33.4437, "longitude": -112.0507, "threshold_minutes": 105,
     "hourly_rate": 1075.0, "max_payout": 19500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@smt.com", "contact_phone": "602-555-2734",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-082", "business_name": "Tempe Co-Working Hub", "business_type": "Co-Working Space",
     "zip_code": "85008", "address": "3730 N Market Blvd", "city": "Phoenix", "state": "AZ",
     "latitude": 33.4273, "longitude": -112.0555, "threshold_minutes": 121,
     "hourly_rate": 525.0, "max_payout": 14000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@tch.com", "contact_phone": "602-555-9693",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-083", "business_name": "Encanto Dental Practice", "business_type": "Dental Office",
     "zip_code": "85012", "address": "2519 S Lake Ave", "city": "Phoenix", "state": "AZ",
     "latitude": 33.441, "longitude": -112.0511, "threshold_minutes": 77,
     "hourly_rate": 800.0, "max_payout": 26500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@edp.com", "contact_phone": "602-555-4232",
     "created_at": now, "updated_at": now}
]
print(f"  🌵 Phoenix, AZ: {len(phoenix_policies)} policies defined")

### 🎸 Austin, TX — 6 Policies

In [ ]:
austin_policies = [
    {"policy_id": "BI-084", "business_name": "South Congress Boutique", "business_type": "Retail",
     "zip_code": "78701", "address": "5726 SE Lincoln Ave", "city": "Austin", "state": "TX",
     "latitude": 30.2901, "longitude": -97.7456, "threshold_minutes": 161,
     "hourly_rate": 475.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@scb.com", "contact_phone": "512-555-1641",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-085", "business_name": "East Austin Taqueria", "business_type": "Restaurant",
     "zip_code": "78702", "address": "4734 NW Union Pl", "city": "Austin", "state": "TX",
     "latitude": 30.2684, "longitude": -97.7376, "threshold_minutes": 121,
     "hourly_rate": 475.0, "max_payout": 12000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@eat.com", "contact_phone": "512-555-6482",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-086", "business_name": "Rainey Street Veterinary Hospital", "business_type": "Veterinary Clinic",
     "zip_code": "78703", "address": "9974 NW Grant Blvd", "city": "Austin", "state": "TX",
     "latitude": 30.2615, "longitude": -97.7555, "threshold_minutes": 64,
     "hourly_rate": 725.0, "max_payout": 19500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@rsv.com", "contact_phone": "512-555-3317",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-087", "business_name": "Domain Seafood Grill", "business_type": "Restaurant",
     "zip_code": "78704", "address": "1488 NE Hill Ct", "city": "Austin", "state": "TX",
     "latitude": 30.2753, "longitude": -97.7617, "threshold_minutes": 67,
     "hourly_rate": 750.0, "max_payout": 20500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@dsg.com", "contact_phone": "512-555-3093",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-088", "business_name": "Zilker Steakhouse", "business_type": "Restaurant",
     "zip_code": "78705", "address": "7037 High Ln", "city": "Austin", "state": "TX",
     "latitude": 30.2921, "longitude": -97.7225, "threshold_minutes": 87,
     "hourly_rate": 1050.0, "max_payout": 29000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@zilkersteakhouse.com", "contact_phone": "512-555-6060",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-089", "business_name": "Mueller Electronics Store", "business_type": "Retail",
     "zip_code": "78721", "address": "2638 N Elm Way", "city": "Austin", "state": "TX",
     "latitude": 30.2805, "longitude": -97.7585, "threshold_minutes": 71,
     "hourly_rate": 875.0, "max_payout": 13500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@mes.com", "contact_phone": "512-555-6517",
     "created_at": now, "updated_at": now}
]
print(f"  🎸 Austin, TX: {len(austin_policies)} policies defined")

### 🎵 Nashville, TN — 6 Policies

In [ ]:
nashville_policies = [
    {"policy_id": "BI-090", "business_name": "Broadway Sushi Bar", "business_type": "Restaurant",
     "zip_code": "37201", "address": "1602 SW Monroe Dr", "city": "Nashville", "state": "TN",
     "latitude": 36.1602, "longitude": -86.7809, "threshold_minutes": 78,
     "hourly_rate": 525.0, "max_payout": 11500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@bsb.com", "contact_phone": "615-555-1436",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-091", "business_name": "The Gulch Veterinary Hospital", "business_type": "Veterinary Clinic",
     "zip_code": "37203", "address": "561 NW Market Ave", "city": "Nashville", "state": "TN",
     "latitude": 36.1495, "longitude": -86.7569, "threshold_minutes": 86,
     "hourly_rate": 625.0, "max_payout": 18500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@tgv.com", "contact_phone": "615-555-3254",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-092", "business_name": "East Nashville Taqueria", "business_type": "Restaurant",
     "zip_code": "37204", "address": "7449 N Queen St", "city": "Nashville", "state": "TN",
     "latitude": 36.1725, "longitude": -86.7997, "threshold_minutes": 94,
     "hourly_rate": 375.0, "max_payout": 11000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@ent.com", "contact_phone": "615-555-5193",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-093", "business_name": "12 South Art Gallery", "business_type": "Art Gallery",
     "zip_code": "37206", "address": "4820 N Lake Way", "city": "Nashville", "state": "TN",
     "latitude": 36.1741, "longitude": -86.7802, "threshold_minutes": 147,
     "hourly_rate": 450.0, "max_payout": 8500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@1sa.com", "contact_phone": "615-555-4906",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-094", "business_name": "Germantown Manufacturing Works", "business_type": "Manufacturing",
     "zip_code": "37207", "address": "8882 E Main Ct", "city": "Nashville", "state": "TN",
     "latitude": 36.1408, "longitude": -86.7648, "threshold_minutes": 71,
     "hourly_rate": 1850.0, "max_payout": 45000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@gmw.com", "contact_phone": "615-555-5772",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-095", "business_name": "Hillsboro Village Cold Storage Facility", "business_type": "Cold Storage",
     "zip_code": "37208", "address": "8676 SE Commerce Ave", "city": "Nashville", "state": "TN",
     "latitude": 36.1662, "longitude": -86.7972, "threshold_minutes": 28,
     "hourly_rate": 2000.0, "max_payout": 64000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@hvc.com", "contact_phone": "615-555-7397",
     "created_at": now, "updated_at": now}
]
print(f"  🎵 Nashville, TN: {len(nashville_policies)} policies defined")

### 🏛️ Washington, DC — 6 Policies

In [ ]:
washington_policies = [
    {"policy_id": "BI-096", "business_name": "Georgetown Seafood Grill", "business_type": "Restaurant",
     "zip_code": "20001", "address": "6434 SW Market Blvd", "city": "Washington", "state": "DC",
     "latitude": 38.9006, "longitude": -77.0174, "threshold_minutes": 86,
     "hourly_rate": 775.0, "max_payout": 13500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@gsg.com", "contact_phone": "202-555-7018",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-097", "business_name": "Dupont Circle Data Center", "business_type": "Data Center",
     "zip_code": "20002", "address": "9037 N Hill Ln", "city": "Washington", "state": "DC",
     "latitude": 38.8861, "longitude": -77.0318, "threshold_minutes": 23,
     "hourly_rate": 4500.0, "max_payout": 103500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@dcd.com", "contact_phone": "202-555-6372",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-098", "business_name": "Capitol Hill Law Offices", "business_type": "Law Office",
     "zip_code": "20003", "address": "7425 NE Oak Ln", "city": "Washington", "state": "DC",
     "latitude": 38.9181, "longitude": -77.0536, "threshold_minutes": 51,
     "hourly_rate": 1875.0, "max_payout": 50500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@chl.com", "contact_phone": "202-555-1939",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-099", "business_name": "Adams Morgan Daycare Center", "business_type": "Childcare Center",
     "zip_code": "20005", "address": "610 Maple Rd", "city": "Washington", "state": "DC",
     "latitude": 38.9079, "longitude": -77.0244, "threshold_minutes": 32,
     "hourly_rate": 875.0, "max_payout": 17000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@amd.com", "contact_phone": "202-555-2101",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-100", "business_name": "Shaw Auto Body Shop", "business_type": "Auto Shop",
     "zip_code": "20006", "address": "7471 SW Commerce Ln", "city": "Washington", "state": "DC",
     "latitude": 38.9013, "longitude": -77.0335, "threshold_minutes": 150,
     "hourly_rate": 575.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@sab.com", "contact_phone": "202-555-3255",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-101", "business_name": "U Street Physical Therapy Clinic", "business_type": "Medical Office",
     "zip_code": "20007", "address": "8276 SE High Pl", "city": "Washington", "state": "DC",
     "latitude": 38.8961, "longitude": -77.0139, "threshold_minutes": 78,
     "hourly_rate": 875.0, "max_payout": 19500.0, "status": "suspended",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@usp.com", "contact_phone": "202-555-5288",
     "created_at": now, "updated_at": now}
]
print(f"  🏛️ Washington, DC: {len(washington_policies)} policies defined")

### 🧊 Minneapolis, MN — 6 Policies

In [ ]:
minneapolis_policies = [
    {"policy_id": "BI-102", "business_name": "North Loop Pizza Kitchen", "business_type": "Restaurant",
     "zip_code": "55401", "address": "1308 N Maple Ln", "city": "Minneapolis", "state": "MN",
     "latitude": 44.9791, "longitude": -93.2515, "threshold_minutes": 85,
     "hourly_rate": 700.0, "max_payout": 13000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@nlp.com", "contact_phone": "612-555-8632",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-103", "business_name": "Uptown Ice Cream Parlor", "business_type": "Ice Cream Shop",
     "zip_code": "55402", "address": "2214 Liberty Ct", "city": "Minneapolis", "state": "MN",
     "latitude": 44.9943, "longitude": -93.2624, "threshold_minutes": 103,
     "hourly_rate": 300.0, "max_payout": 4500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@uic.com", "contact_phone": "612-555-9488",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-104", "business_name": "Northeast Production Studio", "business_type": "Production Studio",
     "zip_code": "55403", "address": "8601 N Sherman Way", "city": "Minneapolis", "state": "MN",
     "latitude": 44.9995, "longitude": -93.2787, "threshold_minutes": 34,
     "hourly_rate": 1600.0, "max_payout": 35500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@nps.com", "contact_phone": "612-555-4216",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-105", "business_name": "Loring Park Bookshop", "business_type": "Retail",
     "zip_code": "55404", "address": "2862 S King Blvd", "city": "Minneapolis", "state": "MN",
     "latitude": 44.9841, "longitude": -93.2593, "threshold_minutes": 261,
     "hourly_rate": 400.0, "max_payout": 5000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@lpb.com", "contact_phone": "612-555-1674",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-106", "business_name": "Whittier Sushi Bar", "business_type": "Restaurant",
     "zip_code": "55405", "address": "3553 Adams Pl", "city": "Minneapolis", "state": "MN",
     "latitude": 45.0002, "longitude": -93.2514, "threshold_minutes": 112,
     "hourly_rate": 850.0, "max_payout": 11000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@wsb.com", "contact_phone": "612-555-9938",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-107", "business_name": "Dinkytown Pharmacy & Wellness", "business_type": "Pharmacy",
     "zip_code": "55406", "address": "7530 S Elm St", "city": "Minneapolis", "state": "MN",
     "latitude": 44.965, "longitude": -93.2496, "threshold_minutes": 63,
     "hourly_rate": 850.0, "max_payout": 15000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@dinkytownpharmacywellness.com", "contact_phone": "612-555-7809",
     "created_at": now, "updated_at": now}
]
print(f"  🧊 Minneapolis, MN: {len(minneapolis_policies)} policies defined")

### 🏭 Detroit, MI — 6 Policies

In [ ]:
detroit_policies = [
    {"policy_id": "BI-108", "business_name": "Corktown Day Spa & Salon", "business_type": "Salon",
     "zip_code": "48201", "address": "8893 NE Main Ave", "city": "Detroit", "state": "MI",
     "latitude": 42.3505, "longitude": -83.0381, "threshold_minutes": 96,
     "hourly_rate": 375.0, "max_payout": 11000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@cds.com", "contact_phone": "313-555-5333",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-109", "business_name": "Midtown Vintage Clothing", "business_type": "Retail",
     "zip_code": "48202", "address": "6039 N Broad Way", "city": "Detroit", "state": "MI",
     "latitude": 42.3087, "longitude": -83.0682, "threshold_minutes": 213,
     "hourly_rate": 325.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@mvc.com", "contact_phone": "313-555-9261",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-110", "business_name": "Eastern Market Pizza Kitchen", "business_type": "Restaurant",
     "zip_code": "48204", "address": "1684 NW Union Ln", "city": "Detroit", "state": "MI",
     "latitude": 42.3559, "longitude": -83.0368, "threshold_minutes": 88,
     "hourly_rate": 625.0, "max_payout": 10000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@emp.com", "contact_phone": "313-555-3363",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-111", "business_name": "Brush Park Fitness Center", "business_type": "Gym",
     "zip_code": "48206", "address": "7861 S High Rd", "city": "Detroit", "state": "MI",
     "latitude": 42.3084, "longitude": -83.0689, "threshold_minutes": 145,
     "hourly_rate": 475.0, "max_payout": 9000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@bpf.com", "contact_phone": "313-555-6373",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-112", "business_name": "Mexicantown Bookshop", "business_type": "Retail",
     "zip_code": "48207", "address": "5611 SE High Ct", "city": "Detroit", "state": "MI",
     "latitude": 42.3559, "longitude": -83.0338, "threshold_minutes": 299,
     "hourly_rate": 325.0, "max_payout": 4500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@mexicantownbookshop.com", "contact_phone": "313-555-5911",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-113", "business_name": "Southwest Law Offices", "business_type": "Law Office",
     "zip_code": "48208", "address": "536 W Queen Rd", "city": "Detroit", "state": "MI",
     "latitude": 42.3401, "longitude": -83.0218, "threshold_minutes": 45,
     "hourly_rate": 2550.0, "max_payout": 48000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@slo.com", "contact_phone": "313-555-1841",
     "created_at": now, "updated_at": now}
]
print(f"  🏭 Detroit, MI: {len(detroit_policies)} policies defined")

### 🏦 Charlotte, NC — 6 Policies

In [ ]:
charlotte_policies = [
    {"policy_id": "BI-114", "business_name": "Uptown Steakhouse", "business_type": "Restaurant",
     "zip_code": "28202", "address": "8111 NW Market Rd", "city": "Charlotte", "state": "NC",
     "latitude": 35.2407, "longitude": -80.8329, "threshold_minutes": 59,
     "hourly_rate": 1175.0, "max_payout": 29500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@uptownsteakhouse.com", "contact_phone": "704-555-5715",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-115", "business_name": "NoDa Dance Academy", "business_type": "Education",
     "zip_code": "28203", "address": "2499 W Hill St", "city": "Charlotte", "state": "NC",
     "latitude": 35.2469, "longitude": -80.8484, "threshold_minutes": 127,
     "hourly_rate": 375.0, "max_payout": 7500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@nda.com", "contact_phone": "704-555-6674",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-116", "business_name": "South End Daycare Center", "business_type": "Childcare Center",
     "zip_code": "28204", "address": "9308 S Lincoln Way", "city": "Charlotte", "state": "NC",
     "latitude": 35.2261, "longitude": -80.858, "threshold_minutes": 59,
     "hourly_rate": 775.0, "max_payout": 15000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@sed.com", "contact_phone": "704-555-4920",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-117", "business_name": "Plaza Midwood Physical Therapy Clinic", "business_type": "Medical Office",
     "zip_code": "28205", "address": "5511 N Lincoln Rd", "city": "Charlotte", "state": "NC",
     "latitude": 35.2134, "longitude": -80.8565, "threshold_minutes": 60,
     "hourly_rate": 725.0, "max_payout": 28500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@pmp.com", "contact_phone": "704-555-1084",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-118", "business_name": "Dilworth Manufacturing Works", "business_type": "Manufacturing",
     "zip_code": "28206", "address": "3367 NW Adams Way", "city": "Charlotte", "state": "NC",
     "latitude": 35.2347, "longitude": -80.8241, "threshold_minutes": 80,
     "hourly_rate": 2175.0, "max_payout": 32500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@dmw.com", "contact_phone": "704-555-1969",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-119", "business_name": "Cherry Seafood Grill", "business_type": "Restaurant",
     "zip_code": "28207", "address": "6946 Sherman Blvd", "city": "Charlotte", "state": "NC",
     "latitude": 35.239, "longitude": -80.8256, "threshold_minutes": 50,
     "hourly_rate": 925.0, "max_payout": 15000.0, "status": "suspended",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@csg.com", "contact_phone": "704-555-9116",
     "created_at": now, "updated_at": now}
]
print(f"  🏦 Charlotte, NC: {len(charlotte_policies)} policies defined")

### 🏖️ San Diego, CA — 6 Policies

In [ ]:
san_diego_policies = [
    {"policy_id": "BI-120", "business_name": "Gaslamp Quarter Art Gallery", "business_type": "Art Gallery",
     "zip_code": "92101", "address": "4385 E Washington Ln", "city": "San Diego", "state": "CA",
     "latitude": 32.7072, "longitude": -117.157, "threshold_minutes": 150,
     "hourly_rate": 350.0, "max_payout": 6500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@gqa.com", "contact_phone": "619-555-2936",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-121", "business_name": "North Park Florist & Garden", "business_type": "Retail",
     "zip_code": "92102", "address": "5195 W Lincoln Ave", "city": "San Diego", "state": "CA",
     "latitude": 32.7158, "longitude": -117.1413, "threshold_minutes": 89,
     "hourly_rate": 425.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@npf.com", "contact_phone": "619-555-4618",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-122", "business_name": "Hillcrest Farm-to-Table", "business_type": "Restaurant",
     "zip_code": "92103", "address": "9098 SE Monroe Rd", "city": "San Diego", "state": "CA",
     "latitude": 32.7134, "longitude": -117.1367, "threshold_minutes": 74,
     "hourly_rate": 925.0, "max_payout": 16500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@hillcrestfarmtotable.com", "contact_phone": "619-555-5054",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-123", "business_name": "Little Italy Fitness Center", "business_type": "Gym",
     "zip_code": "92104", "address": "1379 Monroe Ave", "city": "San Diego", "state": "CA",
     "latitude": 32.6948, "longitude": -117.1638, "threshold_minutes": 207,
     "hourly_rate": 525.0, "max_payout": 13000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@lif.com", "contact_phone": "619-555-9792",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-124", "business_name": "Pacific Beach Day Spa & Salon", "business_type": "Salon",
     "zip_code": "92105", "address": "8108 SW Elm Ct", "city": "San Diego", "state": "CA",
     "latitude": 32.7128, "longitude": -117.1521, "threshold_minutes": 109,
     "hourly_rate": 400.0, "max_payout": 14000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@pbd.com", "contact_phone": "619-555-8429",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-125", "business_name": "Mission Hills Bed & Breakfast", "business_type": "Hotel",
     "zip_code": "92106", "address": "456 Union Way", "city": "San Diego", "state": "CA",
     "latitude": 32.6936, "longitude": -117.163, "threshold_minutes": 68,
     "hourly_rate": 700.0, "max_payout": 15500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@mhb.com", "contact_phone": "619-555-2196",
     "created_at": now, "updated_at": now}
]
print(f"  🏖️ San Diego, CA: {len(san_diego_policies)} policies defined")

### ⛽ Dallas, TX — 6 Policies

In [ ]:
dallas_policies = [
    {"policy_id": "BI-126", "business_name": "Deep Ellum Cold Storage Facility", "business_type": "Cold Storage",
     "zip_code": "75201", "address": "356 N Lake Dr", "city": "Dallas", "state": "TX",
     "latitude": 32.79, "longitude": -96.8204, "threshold_minutes": 23,
     "hourly_rate": 2275.0, "max_payout": 85000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@dec.com", "contact_phone": "214-555-1550",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-127", "business_name": "Bishop Arts Florist & Garden", "business_type": "Retail",
     "zip_code": "75202", "address": "3939 Broad Dr", "city": "Dallas", "state": "TX",
     "latitude": 32.7541, "longitude": -96.785, "threshold_minutes": 88,
     "hourly_rate": 500.0, "max_payout": 11000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@baf.com", "contact_phone": "214-555-9255",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-128", "business_name": "Uptown Bookshop", "business_type": "Retail",
     "zip_code": "75204", "address": "5353 S Lake Way", "city": "Dallas", "state": "TX",
     "latitude": 32.7805, "longitude": -96.7931, "threshold_minutes": 217,
     "hourly_rate": 250.0, "max_payout": 7500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@uptownbookshop.com", "contact_phone": "214-555-7775",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-129", "business_name": "Knox-Henderson Boutique Hotel", "business_type": "Hotel",
     "zip_code": "75205", "address": "7099 Broad Rd", "city": "Dallas", "state": "TX",
     "latitude": 32.7813, "longitude": -96.7852, "threshold_minutes": 39,
     "hourly_rate": 1550.0, "max_payout": 46500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@kbh.com", "contact_phone": "214-555-7244",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-130", "business_name": "Lower Greenville Tutoring Center", "business_type": "Education",
     "zip_code": "75206", "address": "7858 W Lake Dr", "city": "Dallas", "state": "TX",
     "latitude": 32.7592, "longitude": -96.8032, "threshold_minutes": 157,
     "hourly_rate": 375.0, "max_payout": 8500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@lgt.com", "contact_phone": "214-555-8586",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-131", "business_name": "Design District Sushi Bar", "business_type": "Restaurant",
     "zip_code": "75207", "address": "4277 N Park Ln", "city": "Dallas", "state": "TX",
     "latitude": 32.7725, "longitude": -96.7955, "threshold_minutes": 118,
     "hourly_rate": 525.0, "max_payout": 14000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@dds.com", "contact_phone": "214-555-7079",
     "created_at": now, "updated_at": now}
]
print(f"  ⛽ Dallas, TX: {len(dallas_policies)} policies defined")

### 🔔 Philadelphia, PA — 6 Policies

In [ ]:
philadelphia_policies = [
    {"policy_id": "BI-132", "business_name": "Old City Bookshop", "business_type": "Retail",
     "zip_code": "19102", "address": "3725 Madison Blvd", "city": "Philadelphia", "state": "PA",
     "latitude": 39.9461, "longitude": -75.1695, "threshold_minutes": 277,
     "hourly_rate": 375.0, "max_payout": 7500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@ocb.com", "contact_phone": "215-555-7841",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-133", "business_name": "Fishtown Sushi Bar", "business_type": "Restaurant",
     "zip_code": "19103", "address": "5630 SW River Rd", "city": "Philadelphia", "state": "PA",
     "latitude": 39.9466, "longitude": -75.1485, "threshold_minutes": 78,
     "hourly_rate": 550.0, "max_payout": 13500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@fsb.com", "contact_phone": "215-555-3756",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-134", "business_name": "Rittenhouse Comedy Club", "business_type": "Entertainment Venue",
     "zip_code": "19104", "address": "8982 SE Washington St", "city": "Philadelphia", "state": "PA",
     "latitude": 39.9636, "longitude": -75.1658, "threshold_minutes": 136,
     "hourly_rate": 825.0, "max_payout": 11500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@rcc.com", "contact_phone": "215-555-2217",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-135", "business_name": "Northern Liberties Law Offices", "business_type": "Law Office",
     "zip_code": "19106", "address": "1213 Lincoln Blvd", "city": "Philadelphia", "state": "PA",
     "latitude": 39.9279, "longitude": -75.1833, "threshold_minutes": 60,
     "hourly_rate": 2575.0, "max_payout": 36500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@nll.com", "contact_phone": "215-555-9285",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-136", "business_name": "East Passyunk Dance Academy", "business_type": "Education",
     "zip_code": "19107", "address": "8093 SE Main St", "city": "Philadelphia", "state": "PA",
     "latitude": 39.9609, "longitude": -75.1594, "threshold_minutes": 149,
     "hourly_rate": 325.0, "max_payout": 8000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@epd.com", "contact_phone": "215-555-1194",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-137", "business_name": "Manayunk Day Spa & Salon", "business_type": "Salon",
     "zip_code": "19121", "address": "7154 N Franklin Blvd", "city": "Philadelphia", "state": "PA",
     "latitude": 39.9765, "longitude": -75.1893, "threshold_minutes": 92,
     "hourly_rate": 500.0, "max_payout": 9000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@mds.com", "contact_phone": "215-555-9591",
     "created_at": now, "updated_at": now}
]
print(f"  🔔 Philadelphia, PA: {len(philadelphia_policies)} policies defined")

### ☀️ Tampa, FL — 6 Policies

In [ ]:
tampa_policies = [
    {"policy_id": "BI-138", "business_name": "Ybor City Craft Brewery", "business_type": "Brewery",
     "zip_code": "33601", "address": "1360 E Hill Blvd", "city": "Tampa", "state": "FL",
     "latitude": 27.9632, "longitude": -82.4466, "threshold_minutes": 92,
     "hourly_rate": 700.0, "max_payout": 13000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@ycc.com", "contact_phone": "813-555-9856",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-139", "business_name": "SoHo Pet Supply Store", "business_type": "Pet Services",
     "zip_code": "33602", "address": "5766 NE Oak Ave", "city": "Tampa", "state": "FL",
     "latitude": 27.9651, "longitude": -82.4477, "threshold_minutes": 141,
     "hourly_rate": 325.0, "max_payout": 8500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@sps.com", "contact_phone": "813-555-2245",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-140", "business_name": "Seminole Heights Marketing Agency", "business_type": "Professional Services",
     "zip_code": "33603", "address": "4578 SW Franklin Rd", "city": "Tampa", "state": "FL",
     "latitude": 27.9554, "longitude": -82.4338, "threshold_minutes": 81,
     "hourly_rate": 750.0, "max_payout": 16000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@shm.com", "contact_phone": "813-555-6829",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-141", "business_name": "Hyde Park Artisan Bakery", "business_type": "Bakery",
     "zip_code": "33604", "address": "4939 N Franklin Dr", "city": "Tampa", "state": "FL",
     "latitude": 27.9596, "longitude": -82.4492, "threshold_minutes": 95,
     "hourly_rate": 725.0, "max_payout": 11500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@hpa.com", "contact_phone": "813-555-5390",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-142", "business_name": "Channelside Daycare Center", "business_type": "Childcare Center",
     "zip_code": "33605", "address": "1599 SW Main Way", "city": "Tampa", "state": "FL",
     "latitude": 27.9333, "longitude": -82.4627, "threshold_minutes": 34,
     "hourly_rate": 750.0, "max_payout": 21500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@cdc.com", "contact_phone": "813-555-5419",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-143", "business_name": "Westshore Urgent Care Clinic", "business_type": "Medical Office",
     "zip_code": "33606", "address": "530 SE Elm Ln", "city": "Tampa", "state": "FL",
     "latitude": 27.948, "longitude": -82.4545, "threshold_minutes": 33,
     "hourly_rate": 1000.0, "max_payout": 23500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@wuc.com", "contact_phone": "813-555-6315",
     "created_at": now, "updated_at": now}
]
print(f"  ☀️ Tampa, FL: {len(tampa_policies)} policies defined")

### ⛷️ Salt Lake City, UT — 6 Policies

In [ ]:
salt_lake_city_policies = [
    {"policy_id": "BI-144", "business_name": "Sugar House Data Center", "business_type": "Data Center",
     "zip_code": "84101", "address": "8144 NW King Dr", "city": "Salt Lake City", "state": "UT",
     "latitude": 40.7557, "longitude": -111.9, "threshold_minutes": 21,
     "hourly_rate": 3700.0, "max_payout": 70500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@shd.com", "contact_phone": "801-555-5592",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-145", "business_name": "The Avenues Architecture Studio", "business_type": "Professional Services",
     "zip_code": "84102", "address": "3047 Jefferson Ct", "city": "Salt Lake City", "state": "UT",
     "latitude": 40.7824, "longitude": -111.906, "threshold_minutes": 112,
     "hourly_rate": 1000.0, "max_payout": 17500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@taa.com", "contact_phone": "801-555-4434",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-146", "business_name": "Downtown Veterinary Hospital", "business_type": "Veterinary Clinic",
     "zip_code": "84103", "address": "8280 E Maple Way", "city": "Salt Lake City", "state": "UT",
     "latitude": 40.7393, "longitude": -111.8736, "threshold_minutes": 88,
     "hourly_rate": 925.0, "max_payout": 23000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@dvh.com", "contact_phone": "801-555-3525",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-147", "business_name": "Marmalade Sushi Bar", "business_type": "Restaurant",
     "zip_code": "84104", "address": "8855 SE High Pl", "city": "Salt Lake City", "state": "UT",
     "latitude": 40.7583, "longitude": -111.8861, "threshold_minutes": 105,
     "hourly_rate": 675.0, "max_payout": 14000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@msb.com", "contact_phone": "801-555-6890",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-148", "business_name": "9th & 9th Print Shop", "business_type": "Print Shop",
     "zip_code": "84105", "address": "5916 NE Cedar St", "city": "Salt Lake City", "state": "UT",
     "latitude": 40.7677, "longitude": -111.9118, "threshold_minutes": 179,
     "hourly_rate": 675.0, "max_payout": 12500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@9ps.com", "contact_phone": "801-555-8052",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-149", "business_name": "Liberty Park Vintage Clothing", "business_type": "Retail",
     "zip_code": "84106", "address": "9524 NW Lake Rd", "city": "Salt Lake City", "state": "UT",
     "latitude": 40.7613, "longitude": -111.8901, "threshold_minutes": 133,
     "hourly_rate": 250.0, "max_payout": 6500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@lpv.com", "contact_phone": "801-555-8586",
     "created_at": now, "updated_at": now}
]
print(f"  ⛷️ Salt Lake City, UT: {len(salt_lake_city_policies)} policies defined")

### 🌿 Raleigh, NC — 6 Policies

In [ ]:
raleigh_policies = [
    {"policy_id": "BI-150", "business_name": "Downtown Architecture Studio", "business_type": "Professional Services",
     "zip_code": "27601", "address": "9802 Liberty Pl", "city": "Raleigh", "state": "NC",
     "latitude": 35.7561, "longitude": -78.6325, "threshold_minutes": 71,
     "hourly_rate": 925.0, "max_payout": 18000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@das.com", "contact_phone": "919-555-4371",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-151", "business_name": "Glenwood South Comedy Club", "business_type": "Entertainment Venue",
     "zip_code": "27603", "address": "3163 E Commerce Ln", "city": "Raleigh", "state": "NC",
     "latitude": 35.7577, "longitude": -78.6581, "threshold_minutes": 107,
     "hourly_rate": 800.0, "max_payout": 18000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@gsc.com", "contact_phone": "919-555-2867",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-152", "business_name": "Warehouse District Cold Storage Facility", "business_type": "Cold Storage",
     "zip_code": "27604", "address": "6531 NW King Ave", "city": "Raleigh", "state": "NC",
     "latitude": 35.8029, "longitude": -78.6475, "threshold_minutes": 23,
     "hourly_rate": 3000.0, "max_payout": 69500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@wdc.com", "contact_phone": "919-555-8501",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-153", "business_name": "Five Points Dental Practice", "business_type": "Dental Office",
     "zip_code": "27605", "address": "2294 E Market Ave", "city": "Raleigh", "state": "NC",
     "latitude": 35.7946, "longitude": -78.6173, "threshold_minutes": 84,
     "hourly_rate": 775.0, "max_payout": 25000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@fpd.com", "contact_phone": "919-555-2916",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-154", "business_name": "Cameron Village Live Music Venue", "business_type": "Entertainment Venue",
     "zip_code": "27606", "address": "7031 SE Cedar Rd", "city": "Raleigh", "state": "NC",
     "latitude": 35.774, "longitude": -78.6334, "threshold_minutes": 56,
     "hourly_rate": 1050.0, "max_payout": 20500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@cvl.com", "contact_phone": "919-555-3733",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-155", "business_name": "Oakwood Boutique", "business_type": "Retail",
     "zip_code": "27607", "address": "257 E Church Blvd", "city": "Raleigh", "state": "NC",
     "latitude": 35.7967, "longitude": -78.6322, "threshold_minutes": 182,
     "hourly_rate": 375.0, "max_payout": 7500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@oakwoodboutique.com", "contact_phone": "919-555-9281",
     "created_at": now, "updated_at": now}
]
print(f"  🌿 Raleigh, NC: {len(raleigh_policies)} policies defined")

### 🎰 Las Vegas, NV — 6 Policies

In [ ]:
las_vegas_policies = [
    {"policy_id": "BI-156", "business_name": "Downtown Auto Body Shop", "business_type": "Auto Shop",
     "zip_code": "89101", "address": "7115 E Cedar Blvd", "city": "Las Vegas", "state": "NV",
     "latitude": 36.1818, "longitude": -115.1303, "threshold_minutes": 219,
     "hourly_rate": 600.0, "max_payout": 8500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@dab.com", "contact_phone": "702-555-1972",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-157", "business_name": "Arts District Boutique", "business_type": "Retail",
     "zip_code": "89102", "address": "4840 SE King Way", "city": "Las Vegas", "state": "NV",
     "latitude": 36.1718, "longitude": -115.1486, "threshold_minutes": 192,
     "hourly_rate": 400.0, "max_payout": 8000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@adb.com", "contact_phone": "702-555-3221",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-158", "business_name": "Summerlin Veterinary Hospital", "business_type": "Veterinary Clinic",
     "zip_code": "89103", "address": "2274 NW Lake Blvd", "city": "Las Vegas", "state": "NV",
     "latitude": 36.1945, "longitude": -115.1214, "threshold_minutes": 108,
     "hourly_rate": 925.0, "max_payout": 24000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@svh.com", "contact_phone": "702-555-7434",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-159", "business_name": "Henderson Wine Bar", "business_type": "Wine Bar",
     "zip_code": "89104", "address": "1625 W Union Ct", "city": "Las Vegas", "state": "NV",
     "latitude": 36.1763, "longitude": -115.146, "threshold_minutes": 121,
     "hourly_rate": 400.0, "max_payout": 11500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@hwb.com", "contact_phone": "702-555-8449",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-160", "business_name": "Chinatown Art Gallery", "business_type": "Art Gallery",
     "zip_code": "89106", "address": "7611 N Washington Ln", "city": "Las Vegas", "state": "NV",
     "latitude": 36.1836, "longitude": -115.1282, "threshold_minutes": 182,
     "hourly_rate": 325.0, "max_payout": 5000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@cag.com", "contact_phone": "702-555-4568",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-161", "business_name": "Spring Valley Movie Theater", "business_type": "Entertainment Venue",
     "zip_code": "89107", "address": "2239 NW King Rd", "city": "Las Vegas", "state": "NV",
     "latitude": 36.161, "longitude": -115.1506, "threshold_minutes": 87,
     "hourly_rate": 1200.0, "max_payout": 22500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@svm.com", "contact_phone": "702-555-1752",
     "created_at": now, "updated_at": now}
]
print(f"  🎰 Las Vegas, NV: {len(las_vegas_policies)} policies defined")

### 🥩 Kansas City, MO — 6 Policies

In [ ]:
kansas_city_policies = [
    {"policy_id": "BI-162", "business_name": "Crossroads Tech Incubator", "business_type": "Co-Working Space",
     "zip_code": "64101", "address": "5002 S Maple Pl", "city": "Kansas City", "state": "MO",
     "latitude": 39.1112, "longitude": -94.5985, "threshold_minutes": 90,
     "hourly_rate": 525.0, "max_payout": 16000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@cti.com", "contact_phone": "816-555-2120",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-163", "business_name": "Westport Farm-to-Table", "business_type": "Restaurant",
     "zip_code": "64102", "address": "2580 SW Monroe Pl", "city": "Kansas City", "state": "MO",
     "latitude": 39.0829, "longitude": -94.586, "threshold_minutes": 93,
     "hourly_rate": 775.0, "max_payout": 17500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@westportfarmtotable.com", "contact_phone": "816-555-2420",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-164", "business_name": "Country Club Plaza Bed & Breakfast", "business_type": "Hotel",
     "zip_code": "64105", "address": "5252 E Franklin Blvd", "city": "Kansas City", "state": "MO",
     "latitude": 39.1042, "longitude": -94.5844, "threshold_minutes": 117,
     "hourly_rate": 550.0, "max_payout": 19000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@ccp.com", "contact_phone": "816-555-2380",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-165", "business_name": "River Market Recording Studio", "business_type": "Recording Studio",
     "zip_code": "64106", "address": "5042 SE Park Ct", "city": "Kansas City", "state": "MO",
     "latitude": 39.1051, "longitude": -94.5647, "threshold_minutes": 37,
     "hourly_rate": 1550.0, "max_payout": 40000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@rmr.com", "contact_phone": "816-555-7418",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-166", "business_name": "Brookside Yoga & Pilates Studio", "business_type": "Fitness Studio",
     "zip_code": "64108", "address": "4049 SE Church Ave", "city": "Kansas City", "state": "MO",
     "latitude": 39.1144, "longitude": -94.5916, "threshold_minutes": 157,
     "hourly_rate": 325.0, "max_payout": 8000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@bys.com", "contact_phone": "816-555-6916",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-167", "business_name": "Waldo Bookshop", "business_type": "Retail",
     "zip_code": "64109", "address": "2930 N Adams Rd", "city": "Kansas City", "state": "MO",
     "latitude": 39.1166, "longitude": -94.5826, "threshold_minutes": 299,
     "hourly_rate": 275.0, "max_payout": 5500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@waldobookshop.com", "contact_phone": "816-555-2106",
     "created_at": now, "updated_at": now}
]
print(f"  🥩 Kansas City, MO: {len(kansas_city_policies)} policies defined")

### 🏈 Columbus, OH — 6 Policies

In [ ]:
columbus_policies = [
    {"policy_id": "BI-168", "business_name": "Short North Fitness Center", "business_type": "Gym",
     "zip_code": "43201", "address": "4347 S Cedar Ln", "city": "Columbus", "state": "OH",
     "latitude": 39.9535, "longitude": -82.9979, "threshold_minutes": 209,
     "hourly_rate": 300.0, "max_payout": 10000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@snf.com", "contact_phone": "614-555-2934",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-169", "business_name": "German Village Cold Storage Facility", "business_type": "Cold Storage",
     "zip_code": "43202", "address": "1362 SE Church Rd", "city": "Columbus", "state": "OH",
     "latitude": 39.9624, "longitude": -83.0107, "threshold_minutes": 15,
     "hourly_rate": 2500.0, "max_payout": 67000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@gvc.com", "contact_phone": "614-555-1908",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-170", "business_name": "Italian Village Bookshop", "business_type": "Retail",
     "zip_code": "43203", "address": "1989 Lake Ct", "city": "Columbus", "state": "OH",
     "latitude": 39.9441, "longitude": -82.9851, "threshold_minutes": 243,
     "hourly_rate": 375.0, "max_payout": 4500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@ivb.com", "contact_phone": "614-555-1201",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-171", "business_name": "Clintonville Artisan Bakery", "business_type": "Bakery",
     "zip_code": "43205", "address": "679 E Broad Pl", "city": "Columbus", "state": "OH",
     "latitude": 39.9822, "longitude": -83.0149, "threshold_minutes": 75,
     "hourly_rate": 425.0, "max_payout": 11000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@cab.com", "contact_phone": "614-555-5067",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-172", "business_name": "Victorian Village Juice & Smoothie Bar", "business_type": "Juice Bar",
     "zip_code": "43206", "address": "962 SE Cedar Dr", "city": "Columbus", "state": "OH",
     "latitude": 39.9775, "longitude": -82.9952, "threshold_minutes": 173,
     "hourly_rate": 325.0, "max_payout": 4500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@vvj.com", "contact_phone": "614-555-7568",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-173", "business_name": "Franklinton Pharmacy & Wellness", "business_type": "Pharmacy",
     "zip_code": "43210", "address": "7583 E Main Ln", "city": "Columbus", "state": "OH",
     "latitude": 39.9749, "longitude": -83.0083, "threshold_minutes": 49,
     "hourly_rate": 950.0, "max_payout": 20500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@franklintonpharmacywellness.com", "contact_phone": "614-555-5979",
     "created_at": now, "updated_at": now}
]
print(f"  🏈 Columbus, OH: {len(columbus_policies)} policies defined")

### 🏎️ Indianapolis, IN — 6 Policies

In [ ]:
indianapolis_policies = [
    {"policy_id": "BI-174", "business_name": "Broad Ripple Ice Cream Parlor", "business_type": "Ice Cream Shop",
     "zip_code": "46201", "address": "6538 NW Lincoln Ln", "city": "Indianapolis", "state": "IN",
     "latitude": 39.758, "longitude": -86.1406, "threshold_minutes": 87,
     "hourly_rate": 325.0, "max_payout": 4500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@bri.com", "contact_phone": "317-555-5455",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-175", "business_name": "Mass Ave Juice & Smoothie Bar", "business_type": "Juice Bar",
     "zip_code": "46202", "address": "2793 NW State Rd", "city": "Indianapolis", "state": "IN",
     "latitude": 39.7824, "longitude": -86.1539, "threshold_minutes": 149,
     "hourly_rate": 200.0, "max_payout": 4000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@maj.com", "contact_phone": "317-555-2978",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-176", "business_name": "Fountain Square Grocery Market", "business_type": "Grocery Store",
     "zip_code": "46203", "address": "1581 Cedar Way", "city": "Indianapolis", "state": "IN",
     "latitude": 39.766, "longitude": -86.1599, "threshold_minutes": 64,
     "hourly_rate": 775.0, "max_payout": 21000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@fsg.com", "contact_phone": "317-555-6370",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-177", "business_name": "Fletcher Place Accounting Firm", "business_type": "Financial Services",
     "zip_code": "46204", "address": "4362 Hamilton Blvd", "city": "Indianapolis", "state": "IN",
     "latitude": 39.7612, "longitude": -86.1574, "threshold_minutes": 34,
     "hourly_rate": 1300.0, "max_payout": 34500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@fpa.com", "contact_phone": "317-555-8408",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-178", "business_name": "Meridian-Kessler Dance Academy", "business_type": "Education",
     "zip_code": "46205", "address": "295 S Oak Dr", "city": "Indianapolis", "state": "IN",
     "latitude": 39.7665, "longitude": -86.1429, "threshold_minutes": 155,
     "hourly_rate": 400.0, "max_payout": 6000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@mda.com", "contact_phone": "317-555-7563",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-179", "business_name": "Irvington Tech Incubator", "business_type": "Co-Working Space",
     "zip_code": "46208", "address": "1328 E Main Ave", "city": "Indianapolis", "state": "IN",
     "latitude": 39.7476, "longitude": -86.1538, "threshold_minutes": 83,
     "hourly_rate": 775.0, "max_payout": 21500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@iti.com", "contact_phone": "317-555-3110",
     "created_at": now, "updated_at": now}
]
print(f"  🏎️ Indianapolis, IN: {len(indianapolis_policies)} policies defined")

### 🌉 Pittsburgh, PA — 6 Policies

In [ ]:
pittsburgh_policies = [
    {"policy_id": "BI-180", "business_name": "Lawrenceville Event Space", "business_type": "Entertainment Venue",
     "zip_code": "15201", "address": "3688 Main Way", "city": "Pittsburgh", "state": "PA",
     "latitude": 40.435, "longitude": -80.0119, "threshold_minutes": 82,
     "hourly_rate": 1150.0, "max_payout": 21000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@les.com", "contact_phone": "412-555-9674",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-181", "business_name": "Strip District Cold Storage Facility", "business_type": "Cold Storage",
     "zip_code": "15203", "address": "4466 E Broad St", "city": "Pittsburgh", "state": "PA",
     "latitude": 40.4643, "longitude": -80.0159, "threshold_minutes": 26,
     "hourly_rate": 3150.0, "max_payout": 90000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@sdc.com", "contact_phone": "412-555-6813",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-182", "business_name": "Shadyside Manufacturing Works", "business_type": "Manufacturing",
     "zip_code": "15206", "address": "562 S Lincoln Ln", "city": "Pittsburgh", "state": "PA",
     "latitude": 40.4559, "longitude": -79.986, "threshold_minutes": 56,
     "hourly_rate": 1375.0, "max_payout": 44000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@smw.com", "contact_phone": "412-555-9064",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-183", "business_name": "South Side Boutique", "business_type": "Retail",
     "zip_code": "15208", "address": "8725 NW Church St", "city": "Pittsburgh", "state": "PA",
     "latitude": 40.4201, "longitude": -80.0093, "threshold_minutes": 189,
     "hourly_rate": 400.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@ssb.com", "contact_phone": "412-555-3137",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-184", "business_name": "Bloomfield Pharmacy & Wellness", "business_type": "Pharmacy",
     "zip_code": "15210", "address": "2420 SE Adams Ave", "city": "Pittsburgh", "state": "PA",
     "latitude": 40.4578, "longitude": -80.0047, "threshold_minutes": 66,
     "hourly_rate": 875.0, "max_payout": 21000.0, "status": "pending",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@bloomfieldpharmacywellness.com", "contact_phone": "412-555-2188",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-185", "business_name": "Oakland Art Gallery", "business_type": "Art Gallery",
     "zip_code": "15211", "address": "2680 SW Madison Pl", "city": "Pittsburgh", "state": "PA",
     "latitude": 40.4449, "longitude": -80.0041, "threshold_minutes": 167,
     "hourly_rate": 425.0, "max_payout": 9000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@oag.com", "contact_phone": "412-555-3854",
     "created_at": now, "updated_at": now}
]
print(f"  🌉 Pittsburgh, PA: {len(pittsburgh_policies)} policies defined")

### ⚜️ New Orleans, LA — 6 Policies

In [ ]:
new_orleans_policies = [
    {"policy_id": "BI-186", "business_name": "French Quarter Daycare Center", "business_type": "Childcare Center",
     "zip_code": "70112", "address": "7955 S Franklin Dr", "city": "New Orleans", "state": "LA",
     "latitude": 29.945, "longitude": -90.0692, "threshold_minutes": 35,
     "hourly_rate": 950.0, "max_payout": 18000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@fqd.com", "contact_phone": "504-555-2207",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-187", "business_name": "Garden District Co-Working Hub", "business_type": "Co-Working Space",
     "zip_code": "70113", "address": "6251 SE Cedar Ave", "city": "New Orleans", "state": "LA",
     "latitude": 29.9737, "longitude": -90.0928, "threshold_minutes": 94,
     "hourly_rate": 500.0, "max_payout": 13000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@gdc.com", "contact_phone": "504-555-6536",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-188", "business_name": "Marigny Manufacturing Works", "business_type": "Manufacturing",
     "zip_code": "70114", "address": "7213 N Cedar St", "city": "New Orleans", "state": "LA",
     "latitude": 29.9613, "longitude": -90.0749, "threshold_minutes": 83,
     "hourly_rate": 2175.0, "max_payout": 52000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@mmw.com", "contact_phone": "504-555-4367",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-189", "business_name": "Bywater Boutique", "business_type": "Retail",
     "zip_code": "70115", "address": "4972 Lake Ave", "city": "New Orleans", "state": "LA",
     "latitude": 29.9724, "longitude": -90.0695, "threshold_minutes": 144,
     "hourly_rate": 425.0, "max_payout": 10000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@bywaterboutique.com", "contact_phone": "504-555-9043",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-190", "business_name": "Warehouse District Craft Brewery", "business_type": "Brewery",
     "zip_code": "70116", "address": "7969 Park Ave", "city": "New Orleans", "state": "LA",
     "latitude": 29.9305, "longitude": -90.0553, "threshold_minutes": 109,
     "hourly_rate": 425.0, "max_payout": 9000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@wdc.com", "contact_phone": "504-555-3798",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-191", "business_name": "Irish Channel Physical Therapy Clinic", "business_type": "Medical Office",
     "zip_code": "70117", "address": "8113 N Sherman Way", "city": "New Orleans", "state": "LA",
     "latitude": 29.9683, "longitude": -90.0811, "threshold_minutes": 85,
     "hourly_rate": 850.0, "max_payout": 16000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@icp.com", "contact_phone": "504-555-4029",
     "created_at": now, "updated_at": now}
]
print(f"  ⚜️ New Orleans, LA: {len(new_orleans_policies)} policies defined")

### 🍺 Milwaukee, WI — 6 Policies

In [ ]:
milwaukee_policies = [
    {"policy_id": "BI-192", "business_name": "Third Ward Accounting Firm", "business_type": "Financial Services",
     "zip_code": "53202", "address": "9215 NW Union Ct", "city": "Milwaukee", "state": "WI",
     "latitude": 43.0503, "longitude": -87.8911, "threshold_minutes": 56,
     "hourly_rate": 1800.0, "max_payout": 36500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@twa.com", "contact_phone": "414-555-9514",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-193", "business_name": "Bay View Recording Studio", "business_type": "Recording Studio",
     "zip_code": "53203", "address": "1943 S Liberty Blvd", "city": "Milwaukee", "state": "WI",
     "latitude": 43.0489, "longitude": -87.9021, "threshold_minutes": 44,
     "hourly_rate": 1925.0, "max_payout": 25500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@bvr.com", "contact_phone": "414-555-2730",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-194", "business_name": "Walker's Point Dental Practice", "business_type": "Dental Office",
     "zip_code": "53204", "address": "4427 SW Union Rd", "city": "Milwaukee", "state": "WI",
     "latitude": 43.0573, "longitude": -87.8875, "threshold_minutes": 81,
     "hourly_rate": 1100.0, "max_payout": 27000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@wpd.com", "contact_phone": "414-555-3821",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-195", "business_name": "Riverwest Marketing Agency", "business_type": "Professional Services",
     "zip_code": "53205", "address": "5531 Lincoln Dr", "city": "Milwaukee", "state": "WI",
     "latitude": 43.0479, "longitude": -87.9114, "threshold_minutes": 77,
     "hourly_rate": 650.0, "max_payout": 16000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@rma.com", "contact_phone": "414-555-5400",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-196", "business_name": "Brady Street Event Space", "business_type": "Entertainment Venue",
     "zip_code": "53206", "address": "1709 S Grant Dr", "city": "Milwaukee", "state": "WI",
     "latitude": 43.0536, "longitude": -87.916, "threshold_minutes": 117,
     "hourly_rate": 825.0, "max_payout": 18500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@bse.com", "contact_phone": "414-555-8259",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-197", "business_name": "East Side Artisan Bakery", "business_type": "Bakery",
     "zip_code": "53207", "address": "3563 SW Monroe Pl", "city": "Milwaukee", "state": "WI",
     "latitude": 43.0347, "longitude": -87.8992, "threshold_minutes": 109,
     "hourly_rate": 700.0, "max_payout": 10000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@esa.com", "contact_phone": "414-555-1920",
     "created_at": now, "updated_at": now}
]
print(f"  🍺 Milwaukee, WI: {len(milwaukee_policies)} policies defined")

### 🏛️ Sacramento, CA — 6 Policies

In [ ]:
sacramento_policies = [
    {"policy_id": "BI-198", "business_name": "Midtown Urgent Care Clinic", "business_type": "Medical Office",
     "zip_code": "95811", "address": "3487 SE River Pl", "city": "Sacramento", "state": "CA",
     "latitude": 38.6048, "longitude": -121.4765, "threshold_minutes": 31,
     "hourly_rate": 1025.0, "max_payout": 31500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@muc.com", "contact_phone": "916-555-1145",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-199", "business_name": "East Sacramento Movie Theater", "business_type": "Entertainment Venue",
     "zip_code": "95812", "address": "5104 NE Lincoln Way", "city": "Sacramento", "state": "CA",
     "latitude": 38.5867, "longitude": -121.519, "threshold_minutes": 90,
     "hourly_rate": 1250.0, "max_payout": 24500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@esm.com", "contact_phone": "916-555-6713",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-200", "business_name": "Oak Park Fitness Center", "business_type": "Gym",
     "zip_code": "95814", "address": "1776 NW Main Ct", "city": "Sacramento", "state": "CA",
     "latitude": 38.5905, "longitude": -121.4782, "threshold_minutes": 178,
     "hourly_rate": 375.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@opf.com", "contact_phone": "916-555-5890",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-201", "business_name": "Alkali Flat Pet Supply Store", "business_type": "Pet Services",
     "zip_code": "95816", "address": "4378 E Washington Way", "city": "Sacramento", "state": "CA",
     "latitude": 38.606, "longitude": -121.4874, "threshold_minutes": 163,
     "hourly_rate": 275.0, "max_payout": 5500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@afp.com", "contact_phone": "916-555-9475",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-202", "business_name": "Boulevard Park Tech Incubator", "business_type": "Co-Working Space",
     "zip_code": "95817", "address": "5336 Jefferson Blvd", "city": "Sacramento", "state": "CA",
     "latitude": 38.5995, "longitude": -121.4789, "threshold_minutes": 108,
     "hourly_rate": 600.0, "max_payout": 12500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@bpt.com", "contact_phone": "916-555-5843",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-203", "business_name": "Curtis Park Cold Storage Facility", "business_type": "Cold Storage",
     "zip_code": "95818", "address": "3211 SE Market Dr", "city": "Sacramento", "state": "CA",
     "latitude": 38.5934, "longitude": -121.4902, "threshold_minutes": 23,
     "hourly_rate": 3350.0, "max_payout": 66000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@cpc.com", "contact_phone": "916-555-6873",
     "created_at": now, "updated_at": now}
]
print(f"  🏛️ Sacramento, CA: {len(sacramento_policies)} policies defined")

### 🎢 Orlando, FL — 6 Policies

In [ ]:
orlando_policies = [
    {"policy_id": "BI-204", "business_name": "Thornton Park Production Studio", "business_type": "Production Studio",
     "zip_code": "32801", "address": "2997 Market St", "city": "Orlando", "state": "FL",
     "latitude": 28.5522, "longitude": -81.3798, "threshold_minutes": 34,
     "hourly_rate": 1600.0, "max_payout": 55000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@tpp.com", "contact_phone": "407-555-5481",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-205", "business_name": "Mills 50 Bed & Breakfast", "business_type": "Hotel",
     "zip_code": "32803", "address": "1542 NE Maple Pl", "city": "Orlando", "state": "FL",
     "latitude": 28.5515, "longitude": -81.3797, "threshold_minutes": 73,
     "hourly_rate": 800.0, "max_payout": 12000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@m5b.com", "contact_phone": "407-555-8029",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-206", "business_name": "College Park Art Gallery", "business_type": "Art Gallery",
     "zip_code": "32804", "address": "8800 Park St", "city": "Orlando", "state": "FL",
     "latitude": 28.5233, "longitude": -81.3855, "threshold_minutes": 181,
     "hourly_rate": 350.0, "max_payout": 5000.0, "status": "expired",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@cpa.com", "contact_phone": "407-555-4707",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-207", "business_name": "Audubon Park Grocery Market", "business_type": "Grocery Store",
     "zip_code": "32805", "address": "1169 W Hamilton St", "city": "Orlando", "state": "FL",
     "latitude": 28.5184, "longitude": -81.379, "threshold_minutes": 74,
     "hourly_rate": 1050.0, "max_payout": 15500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@apg.com", "contact_phone": "407-555-8469",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-208", "business_name": "Baldwin Park Wine Bar", "business_type": "Wine Bar",
     "zip_code": "32806", "address": "8131 S Commerce Dr", "city": "Orlando", "state": "FL",
     "latitude": 28.535, "longitude": -81.3699, "threshold_minutes": 136,
     "hourly_rate": 500.0, "max_payout": 10500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@bpw.com", "contact_phone": "407-555-1313",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-209", "business_name": "Ivanhoe Village Physical Therapy Clinic", "business_type": "Medical Office",
     "zip_code": "32807", "address": "2117 NE Commerce St", "city": "Orlando", "state": "FL",
     "latitude": 28.5239, "longitude": -81.4031, "threshold_minutes": 59,
     "hourly_rate": 800.0, "max_payout": 30000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@ivp.com", "contact_phone": "407-555-9761",
     "created_at": now, "updated_at": now}
]
print(f"  🎢 Orlando, FL: {len(orlando_policies)} policies defined")

### 🏘️ Richmond, VA — 6 Policies

In [ ]:
richmond_policies = [
    {"policy_id": "BI-210", "business_name": "The Fan Dental Practice", "business_type": "Dental Office",
     "zip_code": "23219", "address": "1558 SW Monroe Ave", "city": "Richmond", "state": "VA",
     "latitude": 37.5624, "longitude": -77.4445, "threshold_minutes": 80,
     "hourly_rate": 950.0, "max_payout": 18000.0, "status": "pending",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@tfd.com", "contact_phone": "804-555-2475",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-211", "business_name": "Carytown Yoga & Pilates Studio", "business_type": "Fitness Studio",
     "zip_code": "23220", "address": "4950 River Rd", "city": "Richmond", "state": "VA",
     "latitude": 37.5208, "longitude": -77.4482, "threshold_minutes": 219,
     "hourly_rate": 300.0, "max_payout": 7500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@cys.com", "contact_phone": "804-555-3862",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-212", "business_name": "Shockoe Bottom Accounting Firm", "business_type": "Financial Services",
     "zip_code": "23221", "address": "1656 Grant St", "city": "Richmond", "state": "VA",
     "latitude": 37.5558, "longitude": -77.4454, "threshold_minutes": 57,
     "hourly_rate": 1250.0, "max_payout": 36000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@sba.com", "contact_phone": "804-555-3793",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-213", "business_name": "Scott's Addition Auto Body Shop", "business_type": "Auto Shop",
     "zip_code": "23222", "address": "5715 NE Cedar Ave", "city": "Richmond", "state": "VA",
     "latitude": 37.5468, "longitude": -77.4171, "threshold_minutes": 162,
     "hourly_rate": 450.0, "max_payout": 10000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@saa.com", "contact_phone": "804-555-8479",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-214", "business_name": "Church Hill Bookshop", "business_type": "Retail",
     "zip_code": "23223", "address": "2832 Pine Blvd", "city": "Richmond", "state": "VA",
     "latitude": 37.5396, "longitude": -77.4399, "threshold_minutes": 214,
     "hourly_rate": 375.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@chb.com", "contact_phone": "804-555-2699",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-215", "business_name": "Jackson Ward Taqueria", "business_type": "Restaurant",
     "zip_code": "23224", "address": "5975 S Queen St", "city": "Richmond", "state": "VA",
     "latitude": 37.5441, "longitude": -77.4391, "threshold_minutes": 143,
     "hourly_rate": 525.0, "max_payout": 9000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@jwt.com", "contact_phone": "804-555-8997",
     "created_at": now, "updated_at": now}
]
print(f"  🏘️ Richmond, VA: {len(richmond_policies)} policies defined")

### 🌺 Honolulu, HI — 6 Policies

In [ ]:
honolulu_policies = [
    {"policy_id": "BI-216", "business_name": "Waikiki Pharmacy & Wellness", "business_type": "Pharmacy",
     "zip_code": "96813", "address": "8248 N Madison St", "city": "Honolulu", "state": "HI",
     "latitude": 21.2863, "longitude": -157.8584, "threshold_minutes": 74,
     "hourly_rate": 775.0, "max_payout": 17500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@waikikipharmacywellness.com", "contact_phone": "808-555-7755",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-217", "business_name": "Chinatown Yoga & Pilates Studio", "business_type": "Fitness Studio",
     "zip_code": "96814", "address": "5090 SE Sherman Ct", "city": "Honolulu", "state": "HI",
     "latitude": 21.2864, "longitude": -157.8531, "threshold_minutes": 124,
     "hourly_rate": 300.0, "max_payout": 4000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@cys.com", "contact_phone": "808-555-8474",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-218", "business_name": "Kakaako Artisan Bakery", "business_type": "Bakery",
     "zip_code": "96815", "address": "9929 SW Church Ln", "city": "Honolulu", "state": "HI",
     "latitude": 21.3227, "longitude": -157.8411, "threshold_minutes": 120,
     "hourly_rate": 525.0, "max_payout": 14500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@kab.com", "contact_phone": "808-555-2596",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-219", "business_name": "Kaimuki Marketing Agency", "business_type": "Professional Services",
     "zip_code": "96816", "address": "9073 N High Ln", "city": "Honolulu", "state": "HI",
     "latitude": 21.2997, "longitude": -157.8516, "threshold_minutes": 108,
     "hourly_rate": 775.0, "max_payout": 10000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@kma.com", "contact_phone": "808-555-3689",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-220", "business_name": "Manoa Electronics Store", "business_type": "Retail",
     "zip_code": "96817", "address": "3172 E Pine Pl", "city": "Honolulu", "state": "HI",
     "latitude": 21.318, "longitude": -157.878, "threshold_minutes": 91,
     "hourly_rate": 550.0, "max_payout": 17000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@mes.com", "contact_phone": "808-555-4122",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-221", "business_name": "Ala Moana Ice Cream Parlor", "business_type": "Ice Cream Shop",
     "zip_code": "96818", "address": "2118 SW Adams Rd", "city": "Honolulu", "state": "HI",
     "latitude": 21.2842, "longitude": -157.8516, "threshold_minutes": 105,
     "hourly_rate": 275.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@ami.com", "contact_phone": "808-555-9172",
     "created_at": now, "updated_at": now}
]
print(f"  🌺 Honolulu, HI: {len(honolulu_policies)} policies defined")

### 🐻 Anchorage, AK — 6 Policies

In [ ]:
anchorage_policies = [
    {"policy_id": "BI-222", "business_name": "Downtown Auto Body Shop", "business_type": "Auto Shop",
     "zip_code": "99501", "address": "4784 Sherman Pl", "city": "Anchorage", "state": "AK",
     "latitude": 61.214, "longitude": -149.8879, "threshold_minutes": 194,
     "hourly_rate": 525.0, "max_payout": 11500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@dab.com", "contact_phone": "907-555-1414",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-223", "business_name": "Spenard Pharmacy & Wellness", "business_type": "Pharmacy",
     "zip_code": "99502", "address": "4795 S Jefferson Way", "city": "Anchorage", "state": "AK",
     "latitude": 61.2031, "longitude": -149.8878, "threshold_minutes": 69,
     "hourly_rate": 650.0, "max_payout": 18500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@spenardpharmacywellness.com", "contact_phone": "907-555-5835",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-224", "business_name": "Midtown Sushi Bar", "business_type": "Restaurant",
     "zip_code": "99503", "address": "9305 SE Grant Dr", "city": "Anchorage", "state": "AK",
     "latitude": 61.2338, "longitude": -149.9202, "threshold_minutes": 73,
     "hourly_rate": 800.0, "max_payout": 10500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@msb.com", "contact_phone": "907-555-4892",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-225", "business_name": "South Addition Art Gallery", "business_type": "Art Gallery",
     "zip_code": "99504", "address": "6449 SE Liberty Ln", "city": "Anchorage", "state": "AK",
     "latitude": 61.2075, "longitude": -149.8921, "threshold_minutes": 185,
     "hourly_rate": 450.0, "max_payout": 10000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@saa.com", "contact_phone": "907-555-1212",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-226", "business_name": "Fairview Wine Bar", "business_type": "Wine Bar",
     "zip_code": "99507", "address": "6125 E Adams Ct", "city": "Anchorage", "state": "AK",
     "latitude": 61.2321, "longitude": -149.8818, "threshold_minutes": 137,
     "hourly_rate": 425.0, "max_payout": 11500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "admin@fwb.com", "contact_phone": "907-555-5819",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-227", "business_name": "Mountain View Tutoring Center", "business_type": "Education",
     "zip_code": "99508", "address": "6280 W Oak Pl", "city": "Anchorage", "state": "AK",
     "latitude": 61.2024, "longitude": -149.9166, "threshold_minutes": 147,
     "hourly_rate": 350.0, "max_payout": 10000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@mvt.com", "contact_phone": "907-555-3131",
     "created_at": now, "updated_at": now}
]
print(f"  🐻 Anchorage, AK: {len(anchorage_policies)} policies defined")

### 🥔 Boise, ID — 6 Policies

In [ ]:
boise_policies = [
    {"policy_id": "BI-228", "business_name": "Downtown Sushi Bar", "business_type": "Restaurant",
     "zip_code": "83701", "address": "8951 Madison Way", "city": "Boise", "state": "ID",
     "latitude": 43.6358, "longitude": -116.1801, "threshold_minutes": 62,
     "hourly_rate": 775.0, "max_payout": 13000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@dsb.com", "contact_phone": "208-555-9426",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-229", "business_name": "North End Accounting Firm", "business_type": "Financial Services",
     "zip_code": "83702", "address": "1449 SE Pine Blvd", "city": "Boise", "state": "ID",
     "latitude": 43.6048, "longitude": -116.1947, "threshold_minutes": 55,
     "hourly_rate": 1700.0, "max_payout": 41000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "info@nea.com", "contact_phone": "208-555-2072",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-230", "business_name": "Hyde Park Day Spa & Salon", "business_type": "Salon",
     "zip_code": "83703", "address": "8910 N Hamilton Pl", "city": "Boise", "state": "ID",
     "latitude": 43.6332, "longitude": -116.2061, "threshold_minutes": 116,
     "hourly_rate": 550.0, "max_payout": 13000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@hpd.com", "contact_phone": "208-555-1389",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-231", "business_name": "West End Pet Supply Store", "business_type": "Pet Services",
     "zip_code": "83704", "address": "1254 W King Dr", "city": "Boise", "state": "ID",
     "latitude": 43.5937, "longitude": -116.2184, "threshold_minutes": 149,
     "hourly_rate": 400.0, "max_payout": 5000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@wep.com", "contact_phone": "208-555-7357",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-232", "business_name": "Boise Bench Movie Theater", "business_type": "Entertainment Venue",
     "zip_code": "83705", "address": "1136 N Adams St", "city": "Boise", "state": "ID",
     "latitude": 43.5908, "longitude": -116.1936, "threshold_minutes": 88,
     "hourly_rate": 1075.0, "max_payout": 21500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@bbm.com", "contact_phone": "208-555-9238",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-233", "business_name": "Southeast Boise Pharmacy & Wellness", "business_type": "Pharmacy",
     "zip_code": "83706", "address": "6349 SW Union St", "city": "Boise", "state": "ID",
     "latitude": 43.6203, "longitude": -116.2132, "threshold_minutes": 51,
     "hourly_rate": 725.0, "max_payout": 19000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@sbp.com", "contact_phone": "208-555-9740",
     "created_at": now, "updated_at": now}
]
print(f"  🥔 Boise, ID: {len(boise_policies)} policies defined")

### 🎈 Albuquerque, NM — 6 Policies

In [ ]:
albuquerque_policies = [
    {"policy_id": "BI-234", "business_name": "Old Town Day Spa & Salon", "business_type": "Salon",
     "zip_code": "87101", "address": "1243 NW Sherman Dr", "city": "Albuquerque", "state": "NM",
     "latitude": 35.0658, "longitude": -106.6426, "threshold_minutes": 164,
     "hourly_rate": 500.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@otd.com", "contact_phone": "505-555-8383",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-235", "business_name": "Nob Hill Florist & Garden", "business_type": "Retail",
     "zip_code": "87102", "address": "5282 Liberty Blvd", "city": "Albuquerque", "state": "NM",
     "latitude": 35.0886, "longitude": -106.6471, "threshold_minutes": 81,
     "hourly_rate": 500.0, "max_payout": 9000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "front.desk@nhf.com", "contact_phone": "505-555-5033",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-236", "business_name": "Downtown Urgent Care Clinic", "business_type": "Medical Office",
     "zip_code": "87104", "address": "5532 SW Sherman Rd", "city": "Albuquerque", "state": "NM",
     "latitude": 35.0812, "longitude": -106.6334, "threshold_minutes": 55,
     "hourly_rate": 1050.0, "max_payout": 35000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@duc.com", "contact_phone": "505-555-6877",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-237", "business_name": "EDo Veterinary Hospital", "business_type": "Veterinary Clinic",
     "zip_code": "87105", "address": "2393 E Sherman Ct", "city": "Albuquerque", "state": "NM",
     "latitude": 35.0682, "longitude": -106.6451, "threshold_minutes": 109,
     "hourly_rate": 725.0, "max_payout": 19000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@evh.com", "contact_phone": "505-555-2252",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-238", "business_name": "Barelas Fitness Center", "business_type": "Gym",
     "zip_code": "87106", "address": "6737 Adams Dr", "city": "Albuquerque", "state": "NM",
     "latitude": 35.0848, "longitude": -106.6738, "threshold_minutes": 213,
     "hourly_rate": 325.0, "max_payout": 6000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@bfc.com", "contact_phone": "505-555-9904",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-239", "business_name": "Wells Park Law Offices", "business_type": "Law Office",
     "zip_code": "87107", "address": "9760 S Madison Way", "city": "Albuquerque", "state": "NM",
     "latitude": 35.0982, "longitude": -106.6571, "threshold_minutes": 58,
     "hourly_rate": 2800.0, "max_payout": 51500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@wpl.com", "contact_phone": "505-555-9210",
     "created_at": now, "updated_at": now}
]
print(f"  🎈 Albuquerque, NM: {len(albuquerque_policies)} policies defined")

### 🌸 Charleston, SC — 6 Policies

In [ ]:
charleston_policies = [
    {"policy_id": "BI-240", "business_name": "King Street Medical Center", "business_type": "Medical Office",
     "zip_code": "29401", "address": "5753 Church Rd", "city": "Charleston", "state": "SC",
     "latitude": 32.7626, "longitude": -79.9358, "threshold_minutes": 46,
     "hourly_rate": 1250.0, "max_payout": 49500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@ksm.com", "contact_phone": "843-555-8207",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-241", "business_name": "Harleston Village Dental Practice", "business_type": "Dental Office",
     "zip_code": "29403", "address": "9963 E Lincoln Ave", "city": "Charleston", "state": "SC",
     "latitude": 32.7655, "longitude": -79.9486, "threshold_minutes": 55,
     "hourly_rate": 825.0, "max_payout": 22500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@hvd.com", "contact_phone": "843-555-6410",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-242", "business_name": "French Quarter Electronics Store", "business_type": "Retail",
     "zip_code": "29405", "address": "6832 Main Dr", "city": "Charleston", "state": "SC",
     "latitude": 32.7755, "longitude": -79.9266, "threshold_minutes": 106,
     "hourly_rate": 550.0, "max_payout": 17000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@fqe.com", "contact_phone": "843-555-3639",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-243", "business_name": "Cannonborough Architecture Studio", "business_type": "Professional Services",
     "zip_code": "29407", "address": "3049 High Pl", "city": "Charleston", "state": "SC",
     "latitude": 32.7801, "longitude": -79.9533, "threshold_minutes": 68,
     "hourly_rate": 925.0, "max_payout": 13500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@cas.com", "contact_phone": "843-555-2404",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-244", "business_name": "Wagener Terrace Insurance Agency", "business_type": "Financial Services",
     "zip_code": "29409", "address": "9944 SE Maple Blvd", "city": "Charleston", "state": "SC",
     "latitude": 32.7949, "longitude": -79.9476, "threshold_minutes": 58,
     "hourly_rate": 1125.0, "max_payout": 35000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@wti.com", "contact_phone": "843-555-3747",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-245", "business_name": "Hampton Park Pet Supply Store", "business_type": "Pet Services",
     "zip_code": "29412", "address": "2266 E Pine St", "city": "Charleston", "state": "SC",
     "latitude": 32.7813, "longitude": -79.9527, "threshold_minutes": 166,
     "hourly_rate": 375.0, "max_payout": 9500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@hpp.com", "contact_phone": "843-555-3450",
     "created_at": now, "updated_at": now}
]
print(f"  🌸 Charleston, SC: {len(charleston_policies)} policies defined")

### 🌳 Savannah, GA — 5 Policies

In [ ]:
savannah_policies = [
    {"policy_id": "BI-246", "business_name": "Historic District Ice Cream Parlor", "business_type": "Ice Cream Shop",
     "zip_code": "31401", "address": "9690 SE River Way", "city": "Savannah", "state": "GA",
     "latitude": 32.0717, "longitude": -81.0981, "threshold_minutes": 84,
     "hourly_rate": 325.0, "max_payout": 7000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "manager@hdi.com", "contact_phone": "912-555-6990",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-247", "business_name": "Starland Dental Practice", "business_type": "Dental Office",
     "zip_code": "31404", "address": "8004 SE Cedar Ln", "city": "Savannah", "state": "GA",
     "latitude": 32.0936, "longitude": -81.0955, "threshold_minutes": 84,
     "hourly_rate": 850.0, "max_payout": 25000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "owner@sdp.com", "contact_phone": "912-555-5157",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-248", "business_name": "Midtown Law Offices", "business_type": "Law Office",
     "zip_code": "31405", "address": "2051 SW Madison Dr", "city": "Savannah", "state": "GA",
     "latitude": 32.0758, "longitude": -81.1093, "threshold_minutes": 39,
     "hourly_rate": 2250.0, "max_payout": 62000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "hello@mlo.com", "contact_phone": "912-555-2642",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-249", "business_name": "Ardsley Park Marketing Agency", "business_type": "Professional Services",
     "zip_code": "31406", "address": "952 Cedar Pl", "city": "Savannah", "state": "GA",
     "latitude": 32.0959, "longitude": -81.0855, "threshold_minutes": 73,
     "hourly_rate": 700.0, "max_payout": 18000.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "contact@apm.com", "contact_phone": "912-555-3381",
     "created_at": now, "updated_at": now},
    {"policy_id": "BI-250", "business_name": "Victory Drive Comedy Club", "business_type": "Entertainment Venue",
     "zip_code": "31407", "address": "718 NE Madison Pl", "city": "Savannah", "state": "GA",
     "latitude": 32.0591, "longitude": -81.1051, "threshold_minutes": 120,
     "hourly_rate": 875.0, "max_payout": 14500.0, "status": "active",
     "effective_date": datetime(2026, 1, 1), "expiration_date": None,
     "contact_email": "ops@vdc.com", "contact_phone": "912-555-1941",
     "created_at": now, "updated_at": now}
]
print(f"  🌳 Savannah, GA: {len(savannah_policies)} policies defined")

---
## 🚀 Step 3 — Combine & Write to Delta Table

In [ ]:
# Combine original 11 + all new city lists
all_policies = (
    original_policies
    + seattle_policies
    + portland_policies
    + san_francisco_policies
    + los_angeles_policies
    + new_york_policies
    + chicago_policies
    + miami_policies
    + houston_policies
    + denver_policies
    + boston_policies
    + atlanta_policies
    + phoenix_policies
    + austin_policies
    + nashville_policies
    + washington_policies
    + minneapolis_policies
    + detroit_policies
    + charlotte_policies
    + san_diego_policies
    + dallas_policies
    + philadelphia_policies
    + tampa_policies
    + salt_lake_city_policies
    + raleigh_policies
    + las_vegas_policies
    + kansas_city_policies
    + columbus_policies
    + indianapolis_policies
    + pittsburgh_policies
    + new_orleans_policies
    + milwaukee_policies
    + sacramento_policies
    + orlando_policies
    + richmond_policies
    + honolulu_policies
    + anchorage_policies
    + boise_policies
    + albuquerque_policies
    + charleston_policies
    + savannah_policies
)

print(f"📋 Total policies to load: {len(all_policies)}")
assert len(all_policies) == 250, f"Expected 250, got {len(all_policies)}"

# Deduplicate — skip any policy_ids that already exist
existing_ids = set()
try:
    existing_rows = spark.sql(
        "SELECT policy_id FROM policies WHERE policy_id LIKE 'BI-%'"
    ).collect()
    existing_ids = {row.policy_id for row in existing_rows}
    print(f"   Found {len(existing_ids)} existing policies in table.")
except Exception:
    print("   ⚠️  Could not read existing policies — will load all.")

policies_to_insert = [p for p in all_policies if p["policy_id"] not in existing_ids]
skipped = len(all_policies) - len(policies_to_insert)

if skipped > 0:
    print(f"   ⏭️  Skipping {skipped} policies that already exist.")

if policies_to_insert:
    policies_df = spark.createDataFrame(policies_to_insert, schema=policy_schema)
    policies_df.write.format("delta").mode("append").saveAsTable("policies")
    print(f"\n✅ Loaded {len(policies_to_insert)} policies into Delta table.")
else:
    print("\nℹ️  No new policies to load — all 250 already exist.")


---
## 📊 Step 4 — Verification & Summary

In [ ]:
print("=" * 70)
print("POLICY SUMMARY BY CITY")
print("=" * 70)

display(spark.sql("""
    SELECT
        city, state,
        COUNT(*) AS policy_count,
        SUM(CASE WHEN status = 'active' THEN 1 ELSE 0 END) AS active_count,
        ROUND(AVG(hourly_rate), 0) AS avg_hourly_rate,
        ROUND(SUM(max_payout), 0) AS total_exposure
    FROM policies
    GROUP BY city, state
    ORDER BY policy_count DESC, city
"""))

In [ ]:
print("\n" + "=" * 70)
print("POLICY SUMMARY BY BUSINESS TYPE")
print("=" * 70)

display(spark.sql("""
    SELECT
        business_type,
        COUNT(*) AS count,
        ROUND(AVG(threshold_minutes), 0) AS avg_threshold_min,
        ROUND(AVG(hourly_rate), 0) AS avg_hourly_rate,
        ROUND(AVG(max_payout), 0) AS avg_max_payout
    FROM policies
    WHERE status = 'active'
    GROUP BY business_type
    ORDER BY count DESC, avg_hourly_rate DESC
"""))

In [ ]:
print("\n" + "=" * 70)
print("POLICY STATUS BREAKDOWN")
print("=" * 70)

display(spark.sql("""
    SELECT
        status,
        COUNT(*) AS count,
        ROUND(SUM(max_payout), 0) AS total_exposure
    FROM policies
    GROUP BY status
    ORDER BY count DESC
"""))

In [ ]:
total = spark.sql("SELECT COUNT(*) AS total FROM policies").collect()[0].total
active = spark.sql("SELECT COUNT(*) AS active FROM policies WHERE status='active'").collect()[0].active
cities = spark.sql("SELECT COUNT(DISTINCT city) AS cities FROM policies").collect()[0].cities
states = spark.sql("SELECT COUNT(DISTINCT state) AS states FROM policies").collect()[0].states
types = spark.sql("SELECT COUNT(DISTINCT business_type) AS types FROM policies").collect()[0].types
exposure = spark.sql("SELECT ROUND(SUM(max_payout), 0) AS exposure FROM policies WHERE status='active'").collect()[0].exposure

print(f"\n{'=' * 70}")
print(f"  📊 PORTFOLIO SUMMARY")
print(f"{'=' * 70}")
print(f"  Total Policies:        {total}")
print(f"  Active Policies:       {active}")
print(f"  Unique Cities:         {cities}")
print(f"  States/Territories:    {states}")
print(f"  Business Types:        {types}")
print(f"  Total Exposure:        ${exposure:,.0f}")
print(f"{'=' * 70}")

assert total == 250, f"❌ Expected 250 total policies, got {total}"
print("\n✅ All 250 policies loaded successfully!")

---
## 📋 Full Policy Listing

In [ ]:
display(spark.sql("""
    SELECT
        policy_id, business_name, business_type,
        city, state, zip_code,
        threshold_minutes, hourly_rate, max_payout, status
    FROM policies
    ORDER BY policy_id
"""))